# Importacion de dependencias

In [8916]:
import numpy as np 
import pandas as pd 

# Data Inicial

In [13057]:
liquidacion = '03'
anho = '2026'

# Lectura de Facturación

In [13058]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

print("Abriendo ventana de selección...")

root = tk.Tk()
root.withdraw() 

ruta_archivo = filedialog.askopenfilename(
    title="Selecciona tu Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_facturas = pd.read_excel(
            ruta_archivo, 
            sheet_name='FACTURAS', 
            header=1 
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_facturas)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_facturas = df_facturas.loc[:, ~df_facturas.columns.str.contains('^Unnamed')]

        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        if columna_clave in df_facturas.columns:
            filas_antes = len(df_facturas)
            df_facturas = df_facturas.dropna(subset=[columna_clave])
            filas_despues = len(df_facturas)
            print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_facturas.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_facturas[col].dtype == 'object':
                df_facturas[col] = (
                    df_facturas[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_facturas[col] = pd.to_numeric(df_facturas[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_facturas.head())
        print(df_facturas.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

Abriendo ventana de selección...
🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 963
🧹 Se eliminaron 40 filas (tablas extra al final o vacías).

📊 DATOS LISTOS:
  MOVIL           RUC                        PROPETARIO  \
0   127  1.044282e+10     SONAN KAGUE CHRISTIAN ALFONSO   
1   205  1.021144e+10  BAUTISTA MENDOZA RANDHOP ALFONSO   
2   207  1.043415e+10  YENGLE GONZALES FRANKLIN BERNABÈ   
3   210  1.002890e+10            GUEVARA TORRES ANTONIO   
4   213  1.044043e+10    RODRIGUEZ DONAYRE MIGUEL ANGEL   

   MONTO A EMITIR PROACCION TOTAL NRO DE FACTURA  \
0                        967.5000             PP   
1                       1266.1018     E001 - 876   
2                        522.4265     E001 - 772   
3                        933.1069     E001 - 777   
4                        728.6127    

In [13059]:
df_facturas.dtypes

MOVIL                                object
RUC                                 float64
PROPETARIO                           object
MONTO A EMITIR PROACCION TOTAL      float64
NRO DE FACTURA                       object
MONTO A EMITIR TRANSPORTES TOTAL    float64
NRO DE FACTURA.1                     object
DETALLE PROACCIÓN                    object
OBSERVACIONES PROACCIÓN              object
DETALLE TRANSPORTES                  object
OBSERVACIONES TRANSPORTES            object
MONTO A PAGAR PROACCION             float64
MONTO A PAGAR TRANSPORTES           float64
BANCO                                object
FECHA DE PAGO                       float64
SERVICIO CONTABLE                    object
FACTURA                              object
UNIDAD DE NEGOCIO                    object
dtype: object

In [13060]:
df_facturas.head()

,MOVIL,RUC,PROPETARIO,MONTO A EMITIR PROACCION TOTAL,NRO DE FACTURA,MONTO A EMITIR TRANSPORTES TOTAL,NRO DE FACTURA.1,DETALLE PROACCIÓN,OBSERVACIONES PROACCIÓN,DETALLE TRANSPORTES,OBSERVACIONES TRANSPORTES,MONTO A PAGAR PROACCION,MONTO A PAGAR TRANSPORTES,BANCO,FECHA DE PAGO,SERVICIO CONTABLE,FACTURA,UNIDAD DE NEGOCIO
0,127,1.044282e+10,SONAN KAGUE CHRISTIAN ALFONSO,967.5000,PP,0.0000,NaN,PROCEDE,NaN,NO PROCEDE,NO TIENE PRODUCCIÓN,966.6000,0.0000,INTERBANK,NaN,BÁSICO,PRACTIPAGO,AEROLINEAS
1,205,1.021144e+10,BAUTISTA MENDOZA RANDHOP ALFONSO,1266.1018,E001 - 876,145.7960,E001 - 877,PROCEDE,NaN,PROCEDE,NaN,1028.7018,126.8960,BCP,NaN,No,No,AEROPUERTO
2,207,1.043415e+10,YENGLE GONZALES FRANKLIN BERNABÈ,522.4265,E001 - 772,82.9150,E001 - 773,PROCEDE,NaN,PROCEDE,NaN,147.3712,32.9150,BBVA,NaN,No,No,AEROPUERTO
3,210,1.002890e+10,GUEVARA TORRES ANTONIO,933.1069,E001 - 777,295.5616,E001 - 778,PROCEDE,NaN,PROCEDE,NaN,760.6069,37.9616,BCP,NaN,No,No,AEROPUERTO
4,213,1.044043e+10,RODRIGUEZ DONAYRE MIGUEL ANGEL,728.6127,PP,293.9333,PP,PROCEDE,NaN,PROCEDE,NaN,490.6127,137.8333,BCP,NaN,BÁSICO,PRACTIPAGO,AEROPUERTO


In [13061]:
df_facturas.tail()

,MOVIL,RUC,PROPETARIO,MONTO A EMITIR PROACCION TOTAL,NRO DE FACTURA,MONTO A EMITIR TRANSPORTES TOTAL,NRO DE FACTURA.1,DETALLE PROACCIÓN,OBSERVACIONES PROACCIÓN,DETALLE TRANSPORTES,OBSERVACIONES TRANSPORTES,MONTO A PAGAR PROACCION,MONTO A PAGAR TRANSPORTES,BANCO,FECHA DE PAGO,SERVICIO CONTABLE,FACTURA,UNIDAD DE NEGOCIO
918,9991,1.025651e+10,LEON MEZA JUAN ALFREDO,1405.5404,E001 - 961,226.7545,E001 - 962,PROCEDE,NaN,PROCEDE,NaN,1350.6404,96.7545,BBVA,NaN,No,No,AEROPUERTO
919,9997,1.560855e+10,VALLADARES ROJAS JHONNY GERMIT,20.4000,NaN,0.0000,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO TIENE PRODUCCIÓN,-289.8250,0.0000,BCP,NaN,No,No,AEROPUERTO
920,M8003,1.073941e+10,JOSE DANIEL AGUILAR GABRIEL,16.5100,NaN,22.4250,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO EMITIÓ,16.5100,22.4250,INTERBANK,NaN,No,No,URBANO
921,M8005,1.045335e+10,LUIS JAVIER VALVERDE CONTRERAS,360.0120,NaN,0.0000,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO TIENE PRODUCCIÓN,358.5120,0.0000,BCP,NaN,No,No,URBANO
922,M8002,1.560336e+10,SANCHEZ ABREU RENE ALEJANDRO,13.0000,NaN,13.6500,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO EMITIÓ,5.6500,0.0000,INTERBANK,NaN,No,No,URBANO


In [13062]:
df_facturas.columns

Index(['MOVIL', 'RUC', 'PROPETARIO', 'MONTO A EMITIR PROACCION TOTAL',
       'NRO DE FACTURA', 'MONTO A EMITIR TRANSPORTES TOTAL',
       'NRO DE FACTURA.1', 'DETALLE PROACCIÓN', 'OBSERVACIONES PROACCIÓN',
       'DETALLE TRANSPORTES', 'OBSERVACIONES TRANSPORTES',
       'MONTO A PAGAR PROACCION', 'MONTO A PAGAR TRANSPORTES', 'BANCO',
       'FECHA DE PAGO', 'SERVICIO CONTABLE', 'FACTURA', 'UNIDAD DE NEGOCIO'],
      dtype='object')

In [13063]:
df_facturas.rename(columns={'MONTO A EMITIR PROACCION TOTAL':'MONTO_A_EMITIR_PROACCION_TOTAL', 
'NRO DE FACTURA':'NRO_DE_FACTURA', 'MONTO A EMITIR TRANSPORTES TOTAL':'MONTO_A_EMITIR_TRANSPORTES_TOTAL', 
'NRO DE FACTURA2':'NRO_DE_FACTURA2', 'DETALLE PROACCION':'DETALLE_PROACCION', 'OBSERVACIONES PROACCION':'OBSERVACIONES_PROACCION',
'DETALLE TRANSPORTES':'DETALLE_TRANSPORTES', 'OBSERVACIONES TRANSPORTES':'OBSERVACIONES_TRANSPORTES',
'MONTO A PAGAR PROACCION':'MONTO_A_PAGAR_PROACCION', 'MONTO A PAGAR TRANSPORTES':'MONTO_A_PAGAR_TRANSPORTES', 'FACTURA':'Factura / Rxh'
}, inplace=True)

In [13064]:
df_facturas.dtypes

MOVIL                                object
RUC                                 float64
PROPETARIO                           object
MONTO_A_EMITIR_PROACCION_TOTAL      float64
NRO_DE_FACTURA                       object
MONTO_A_EMITIR_TRANSPORTES_TOTAL    float64
NRO DE FACTURA.1                     object
DETALLE PROACCIÓN                    object
OBSERVACIONES PROACCIÓN              object
DETALLE_TRANSPORTES                  object
OBSERVACIONES_TRANSPORTES            object
MONTO_A_PAGAR_PROACCION             float64
MONTO_A_PAGAR_TRANSPORTES           float64
BANCO                                object
FECHA DE PAGO                       float64
SERVICIO CONTABLE                    object
Factura / Rxh                        object
UNIDAD DE NEGOCIO                    object
dtype: object

In [13065]:
df_facturas[df_facturas["MOVIL"].str.contains("M", na=False)]

,MOVIL,RUC,PROPETARIO,MONTO_A_EMITIR_PROACCION_TOTAL,NRO_DE_FACTURA,MONTO_A_EMITIR_TRANSPORTES_TOTAL,NRO DE FACTURA.1,DETALLE PROACCIÓN,OBSERVACIONES PROACCIÓN,DETALLE_TRANSPORTES,OBSERVACIONES_TRANSPORTES,MONTO_A_PAGAR_PROACCION,MONTO_A_PAGAR_TRANSPORTES,BANCO,FECHA DE PAGO,SERVICIO CONTABLE,Factura / Rxh,UNIDAD DE NEGOCIO
920,M8003,1.073941e+10,JOSE DANIEL AGUILAR GABRIEL,16.510,NaN,22.425,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO EMITIÓ,16.510,22.425,INTERBANK,NaN,No,No,URBANO
921,M8005,1.045335e+10,LUIS JAVIER VALVERDE CONTRERAS,360.012,NaN,0.000,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO TIENE PRODUCCIÓN,358.512,0.000,BCP,NaN,No,No,URBANO
922,M8002,1.560336e+10,SANCHEZ ABREU RENE ALEJANDRO,13.000,NaN,13.650,NaN,NO PROCEDE,NO EMITIÓ,NO PROCEDE,NO EMITIÓ,5.650,0.000,INTERBANK,NaN,No,No,URBANO


In [13066]:
indices_a_borrar = df_facturas[df_facturas["MOVIL"].astype(str).str.contains("M")].index


df_facturas.drop(indices_a_borrar, inplace=True)

#df_facturas[df_facturas["MOVIL"].str.contains("M", na=False)]

In [13067]:
df_facturas['anho'] = anho
df_facturas['LIQ'] = liquidacion
df_facturas['id'] = df_facturas['anho'].astype(str) + df_facturas['LIQ'].astype(str) + df_facturas['MOVIL'].astype(float).astype(int).astype(str)
df_facturas.head()

,MOVIL,RUC,PROPETARIO,MONTO_A_EMITIR_PROACCION_TOTAL,NRO_DE_FACTURA,MONTO_A_EMITIR_TRANSPORTES_TOTAL,NRO DE FACTURA.1,DETALLE PROACCIÓN,OBSERVACIONES PROACCIÓN,DETALLE_TRANSPORTES,...,MONTO_A_PAGAR_PROACCION,MONTO_A_PAGAR_TRANSPORTES,BANCO,FECHA DE PAGO,SERVICIO CONTABLE,Factura / Rxh,UNIDAD DE NEGOCIO,anho,LIQ,id
0,127,1.044282e+10,SONAN KAGUE CHRISTIAN ALFONSO,967.5000,PP,0.0000,NaN,PROCEDE,NaN,NO PROCEDE,...,966.6000,0.0000,INTERBANK,NaN,BÁSICO,PRACTIPAGO,AEROLINEAS,2026,03,202603127
1,205,1.021144e+10,BAUTISTA MENDOZA RANDHOP ALFONSO,1266.1018,E001 - 876,145.7960,E001 - 877,PROCEDE,NaN,PROCEDE,...,1028.7018,126.8960,BCP,NaN,No,No,AEROPUERTO,2026,03,202603205
2,207,1.043415e+10,YENGLE GONZALES FRANKLIN BERNABÈ,522.4265,E001 - 772,82.9150,E001 - 773,PROCEDE,NaN,PROCEDE,...,147.3712,32.9150,BBVA,NaN,No,No,AEROPUERTO,2026,03,202603207
3,210,1.002890e+10,GUEVARA TORRES ANTONIO,933.1069,E001 - 777,295.5616,E001 - 778,PROCEDE,NaN,PROCEDE,...,760.6069,37.9616,BCP,NaN,No,No,AEROPUERTO,2026,03,202603210
4,213,1.044043e+10,RODRIGUEZ DONAYRE MIGUEL ANGEL,728.6127,PP,293.9333,PP,PROCEDE,NaN,PROCEDE,...,490.6127,137.8333,BCP,NaN,BÁSICO,PRACTIPAGO,AEROPUERTO,2026,03,202603213


# Lectura de Descuentos

In [13068]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='DSCTS', 
            header=4
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx


✅ Lectura inicial completada. Filas detectadas: 2630
Flag1
Flag2

📊 DATOS LISTOS:
    MOVIL  Suma de PAGO                         RAZÓN #ID SVC FECHA  \
0  1813.0          30.0  APOYO SOLIDARIO - MOVIL 1434     NaN   NaT   
1  1402.0          10.0  APOYO SOLIDARIO - MOVIL 1434     NaN   NaT   
2  9645.0          10.0  APOYO SOLIDARIO - MOVIL 1434     NaN   NaT   
3  9834.0          10.0  APOYO SOLIDARIO - MOVIL 1434     NaN   NaT   
4  9982.0          20.0  APOYO SOLIDARIO - MOVIL 1434     NaN   NaT   

   TARIFA BRUTA            SUB DETALLE UNIDAD DE NEGOCIO  MOVIL.1  \
0           NaN  Regularización de Svc            URBANO    692.0   
1           NaN  Regularización de Svc            URBANO    692.0   
2           NaN  Regularización de Svc            URBANO    692.0   
3           NaN  Regularización de Svc        AEROLINEAS   3634.0   
4           NaN  Regularización de Svc        AEROPUERTO   9921.0   

   TARIFA BRUTA.1  ... MOVIL.4  Suma de DSCT.1                RAZÓN.1  \
0  

In [13069]:
df_dscts.columns

Index(['MOVIL', 'Suma de PAGO', 'RAZÓN', '#ID SVC', 'FECHA', 'TARIFA BRUTA',
       'SUB DETALLE', 'UNIDAD DE NEGOCIO', 'MOVIL.1', 'TARIFA BRUTA.1',
       'Suma de DSCT', 'MONTO', 'SUB DETALLE.1', 'MOVIL.2', 'SERVICIO',
       'MONTO.1', 'FECHA.1', 'MOVIL.3', 'SERVICIO.1', 'MONTO.2', 'MOVIL.4',
       'Suma de DSCT.1', 'RAZÓN.1', 'MOVIL.5', 'RAZÓN.2', 'MOVIL.6',
       'Suma de DSCT.2', 'RAZÓN.3',
       'DSCTO. PEX DE LA SEMANA #01 DEL 29 DICIEMBRE AL 04 ENERO',
       'FECHA DE PAGO FLOTA AEROPUERTO Y URBANO:  21 DE ENERO     */*     FECHA DE PAGO FLOTA AEROLINEAS: 27 DE ENERO'],
      dtype='object')

In [13070]:

df_dscts_1 = df_dscts.iloc[:,0:7]
df_dscts_1.head()

,MOVIL,Suma de PAGO,RAZÓN,#ID SVC,FECHA,TARIFA BRUTA,SUB DETALLE
0,1813.0,30.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
1,1402.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
2,9645.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
3,9834.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
4,9982.0,20.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc


In [13071]:
df_dscts_1.rename(columns={'Suma de PAGO':'Suma_de_Pago', 'RAZÓN':'RAZON', '#ID SVC':'Nro_Vale_SVS', 
'TARIFA BRUTA':'TARIFA_BRUTA', 'SUB DETALLE':'SUB_DETALLE'}, inplace=True)
df_dscts_1.head()


,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_SVS,FECHA,TARIFA_BRUTA,SUB_DETALLE
0,1813.0,30.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
1,1402.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
2,9645.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
3,9834.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
4,9982.0,20.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc


In [13072]:
df_dscts_1.dropna(subset=['MOVIL'], inplace=True)
df_dscts_1.head()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_SVS,FECHA,TARIFA_BRUTA,SUB_DETALLE
0,1813.0,30.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
1,1402.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
2,9645.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
3,9834.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc
4,9982.0,20.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc


In [13073]:

df_dscts_2 = df_dscts.iloc[:,7:13]
df_dscts_2.reset_index(drop=True, inplace=True)
df_dscts_2.head()

,UNIDAD DE NEGOCIO,MOVIL.1,TARIFA BRUTA.1,Suma de DSCT,MONTO,SUB DETALLE.1
0,URBANO,692.0,2.0,01 –PortaFotocheck,2.0,Uniforme
1,URBANO,692.0,10.0,01 - Corbata,10.0,Uniforme
2,URBANO,692.0,9.0,01 – Cargador,9.0,Uniforme
3,AEROLINEAS,3634.0,10.0,01 –Corbata,10.0,Uniforme
4,AEROPUERTO,9921.0,20.0,02 - Corbata,20.0,Uniforme


In [13074]:
df_dscts_2.rename(columns={'UNIDAD DE NEGOCIO':'UNIDAD_DE_NEGOCIO','MOVIL.1':'MOVIL','TARIFA BRUTA.1':'Suma_de_Pago',
'Suma de DSCT':'RAZON', 'MONTO':'Nro_Vale_SVS', 'SUB DETALLE.1':'SUB_DETALLE'},inplace=True)
df_dscts_2.dropna(subset=["MOVIL"], inplace=True)
df_dscts_2.head()

,UNIDAD_DE_NEGOCIO,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_SVS,SUB_DETALLE
0,URBANO,692.0,2.0,01 –PortaFotocheck,2.0,Uniforme
1,URBANO,692.0,10.0,01 - Corbata,10.0,Uniforme
2,URBANO,692.0,9.0,01 – Cargador,9.0,Uniforme
3,AEROLINEAS,3634.0,10.0,01 –Corbata,10.0,Uniforme
4,AEROPUERTO,9921.0,20.0,02 - Corbata,20.0,Uniforme


In [13075]:

df_dscts_3 = df_dscts.iloc[:,13:17]
df_dscts_3.reset_index(drop=True, inplace=True)
df_dscts_3.head()

,MOVIL.2,SERVICIO,MONTO.1,FECHA.1
0,NaN,Comisión Urbano,NaN,NaN
1,NaN,Comisión Urbano,NaN,NaN
2,NaN,Comisión Urbano,NaN,NaN
3,NaN,Comisión Urbano,NaN,NaN
4,NaN,Comisión Urbano,NaN,NaN


In [13076]:
df_dscts_3.rename(columns={'MOVIL.2':'MOVIL', 'SERVICIO':'RAZON', 'MONTO.1':'Suma_de_Pago', 'FECHA.1':'FECHA'}, inplace=True)
df_dscts_3.dropna(subset=['MOVIL'], inplace=True)
df_dscts_3.head()

,MOVIL,RAZON,Suma_de_Pago,FECHA


In [13077]:
df_dscts_4 = df_dscts.iloc[:,17:20]
df_dscts_4.reset_index(drop=True, inplace=True)
df_dscts_4

,MOVIL.3,SERVICIO.1,MONTO.2
0,735.0,FONDO DE AHORRO INDIVIDUAL,200.0
1,950.0,FONDO DE AHORRO INDIVIDUAL,200.0
2,1406.0,FONDO DE AHORRO INDIVIDUAL,200.0
3,1416.0,FONDO DE AHORRO INDIVIDUAL,200.0
4,1458.0,FONDO DE AHORRO INDIVIDUAL,200.0
...,...,...,...
2625,NaN,NaN,NaN
2626,NaN,NaN,NaN
2627,NaN,NaN,NaN
2628,NaN,NaN,NaN


In [13078]:
df_dscts_4.rename(columns={'MOVIL.3':'MOVIL', 'SERVICIO.1':'RAZON', 'MONTO.2':'Suma_de_Pago'}, inplace=True)
df_dscts_4.dropna(subset=['MOVIL'], inplace=True)
df_dscts_4.head()

,MOVIL,RAZON,Suma_de_Pago
0,735.0,FONDO DE AHORRO INDIVIDUAL,200.0
1,950.0,FONDO DE AHORRO INDIVIDUAL,200.0
2,1406.0,FONDO DE AHORRO INDIVIDUAL,200.0
3,1416.0,FONDO DE AHORRO INDIVIDUAL,200.0
4,1458.0,FONDO DE AHORRO INDIVIDUAL,200.0


In [13079]:
df_dscts_5 = df_dscts.iloc[:,25:28]
df_dscts_5.reset_index(drop=True, inplace=True)
df_dscts_5

,MOVIL.6,Suma de DSCT.2,RAZÓN.3
0,3940.0,6.0,SERVICIO CONTABLE
1,9224.0,6.0,SERVICIO CONTABLE
2,2186.0,6.0,SERVICIO CONTABLE
3,2410.0,6.0,SERVICIO CONTABLE
4,515.0,6.0,SERVICIO CONTABLE
...,...,...,...
2625,NaN,NaN,NaN
2626,NaN,NaN,NaN
2627,NaN,NaN,NaN
2628,NaN,NaN,NaN


In [13080]:
df_dscts_5.rename(columns={'MOVIL.6':'MOVIL', 'Suma de DSCT.2':'Suma_de_Pago', 'RAZÓN.3':'RAZON'}, inplace=True)
df_dscts_5.dropna(subset=['MOVIL'], inplace=True)
df_dscts_5.head()

,MOVIL,Suma_de_Pago,RAZON
0,3940.0,6.0,SERVICIO CONTABLE
1,9224.0,6.0,SERVICIO CONTABLE
2,2186.0,6.0,SERVICIO CONTABLE
3,2410.0,6.0,SERVICIO CONTABLE
4,515.0,6.0,SERVICIO CONTABLE


In [13081]:
df_dscts_6 = df_dscts.iloc[:,20:23]
df_dscts_6.reset_index(drop=True, inplace=True)
df_dscts_6

,MOVIL.4,Suma de DSCT.1,RAZÓN.1
0,717.0,3.5,MEMBRESIA SEMANAL PEX
1,210.0,3.5,MEMBRESIA SEMANAL PEX
2,772.0,3.5,MEMBRESIA SEMANAL PEX
3,230.0,3.5,MEMBRESIA SEMANAL PEX
4,1524.0,3.5,MEMBRESIA SEMANAL PEX
...,...,...,...
2625,NaN,NaN,NaN
2626,NaN,NaN,NaN
2627,NaN,NaN,NaN
2628,NaN,NaN,NaN


In [13082]:
df_dscts_6.rename(columns={'MOVIL.4':'MOVIL', 'Suma de DSCT.1':'Suma_de_Pago', 'RAZÓN.1':'RAZON'}, inplace=True)
df_dscts_6.dropna(subset=['MOVIL'], inplace=True)
df_dscts_6.head()

,MOVIL,Suma_de_Pago,RAZON
0,717.0,3.5,MEMBRESIA SEMANAL PEX
1,210.0,3.5,MEMBRESIA SEMANAL PEX
2,772.0,3.5,MEMBRESIA SEMANAL PEX
3,230.0,3.5,MEMBRESIA SEMANAL PEX
4,1524.0,3.5,MEMBRESIA SEMANAL PEX


In [13083]:
df_dscts_tot = pd.concat([df_dscts_1, df_dscts_2, df_dscts_3, df_dscts_4, df_dscts_5, df_dscts_6], axis=0, ignore_index=True, sort=False)
df_dscts_tot.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18092\4194719742.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dscts_tot = pd.concat([df_dscts_1, df_dscts_2, df_dscts_3, df_dscts_4, df_dscts_5, df_dscts_6], axis=0, ignore_index=True, sort=False)


,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_SVS,FECHA,TARIFA_BRUTA,SUB_DETALLE,UNIDAD_DE_NEGOCIO
0,1813.0,30.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN
1,1402.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN
2,9645.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN
3,9834.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN
4,9982.0,20.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN


In [13084]:
df_dscts_tot.tail()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_SVS,FECHA,TARIFA_BRUTA,SUB_DETALLE,UNIDAD_DE_NEGOCIO
1474,1502.0,3.5,MEMBRESIA SEMANAL PEX,NaN,NaT,NaN,NaN,NaN
1475,1422.0,3.5,MEMBRESIA SEMANAL PEX,NaN,NaT,NaN,NaN,NaN
1476,9382.0,3.5,MEMBRESIA SEMANAL PEX,NaN,NaT,NaN,NaN,NaN
1477,1453.0,3.5,MEMBRESIA SEMANAL PEX,NaN,NaT,NaN,NaN,NaN
1478,4070.0,3.5,MEMBRESIA SEMANAL PEX,NaN,NaT,NaN,NaN,NaN


In [13085]:
df_dscts_tot.columns.to_list()

['MOVIL',
 'Suma_de_Pago',
 'RAZON',
 'Nro_Vale_SVS',
 'FECHA',
 'TARIFA_BRUTA',
 'SUB_DETALLE',
 'UNIDAD_DE_NEGOCIO']

In [13086]:
df_dscts_tot.dropna(subset=['MOVIL'], inplace=True)

In [13087]:

indices_a_borrar = df_dscts_tot[df_dscts_tot["MOVIL"].astype(str).str.contains("M", case=False, na=False)].index

# 2. Borramos
df_dscts_tot.drop(indices_a_borrar, inplace=True)


In [13088]:
df_dscts_tot['anho'] = anho
df_dscts_tot['LIQ'] = liquidacion
df_dscts_tot['id'] = df_dscts_tot['anho'].astype(str) + df_dscts_tot['LIQ'].astype(str) + df_dscts_tot['MOVIL'].astype(float).astype(int).astype(str)
df_dscts_tot.head()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_SVS,FECHA,TARIFA_BRUTA,SUB_DETALLE,UNIDAD_DE_NEGOCIO,anho,LIQ,id
0,1813.0,30.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN,2026,03,2026031813
1,1402.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN,2026,03,2026031402
2,9645.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN,2026,03,2026039645
3,9834.0,10.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN,2026,03,2026039834
4,9982.0,20.0,APOYO SOLIDARIO - MOVIL 1434,NaN,NaT,NaN,Regularización de Svc,NaN,2026,03,2026039982


In [13089]:
# Convertimos a número y lo que no se pueda (NaN) lo volvemos 0
df_dscts_tot['Suma_de_Pago'] = pd.to_numeric(df_dscts_tot['Suma_de_Pago'], errors='coerce').fillna(0)
df_dscts_tot['TARIFA_BRUTA'] = pd.to_numeric(df_dscts_tot['TARIFA_BRUTA'], errors='coerce').fillna(0)
df_dscts_tot['MOVIL'] = pd.to_numeric(df_dscts_tot['MOVIL'], errors='coerce').fillna(0)

# El campo 'id' y 'anho' a veces se leen como texto, aseguremos que sean consistentes
df_dscts_tot['anho'] = df_dscts_tot['anho'].astype(str).str.replace('.0', '', regex=False)
df_dscts_tot['id'] = df_dscts_tot['id'].astype(str).str.replace('.0', '', regex=False)

# Lectura de Matriz

In [13090]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='MATRIZ', 
            header=4
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        if columna_clave in df_dscts.columns:
            filas_antes = len(df_dscts)
            df_dscts = df_dscts.dropna(subset=[columna_clave])
            filas_despues = len(df_dscts)
            print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 17438
Flag1
Flag2

📊 DATOS LISTOS:
  Movil N° Servicio                Fecha Cliente Unidad de Negocio  \
0  1410  B012-70046  2026-01-12 00:00:00    KUSI            ORIGEN   
1   794  B012-70048  2026-01-12 00:00:00    KUSI            ORIGEN   
2  3815  B012-70045  2026-01-12 00:00:00    KUSI            ORIGEN   
3  9656  B012-70044  2026-01-12 00:00:00    KUSI            ORIGEN   
4   879  B012-70043  2026-01-12 00:00:00    KUSI            ORIGEN   

  Tipo de Servicio Tipo de pago  Peaje  Monto a Liquidar  Total Servicio  \
0           TICKET      Crédito    0.0              60.0            60.0   
1           TICKET      Crédito    0.0              50.0            50.0   
2           TICKET      Crédito    0.0              75.0            75.0   
3    

In [13091]:
filas_vacias = df_dscts.isnull().all(axis=1)

if filas_vacias.any():
    # 2. Encontramos el índice de la PRIMERA fila que cumple la condición
    primer_indice_vacio = filas_vacias.idxmax()
    
    # 3. Cortamos el DataFrame: mantenemos todo lo que esté ANTES de ese índice
    df_dscts = df_dscts.iloc[:primer_indice_vacio]
    print(f"✂️ Corte realizado: Se detectó fin de tabla en la fila {primer_indice_vacio}")
else:
    print("ℹ️ No se detectaron filas completamente vacías para cortar.")

✂️ Corte realizado: Se detectó fin de tabla en la fila 15335


In [13092]:
df_dscts.tail()

,Movil,N° Servicio,Fecha,Cliente,Unidad de Negocio,Tipo de Servicio,Tipo de pago,Peaje,Monto a Liquidar,Total Servicio,Comisión,Comisión Aeropuerto,Total para conductor (sin peaje),Comisión Directo,Comisión Conductor,Flota
15330,4049,9869893,2026-01-18 00:00:00,sky tripulante,SKY,Empresa,Crédito,0.0,54.0,54.0,0.25,0.0,40.50,0.25,0.75,AEROLINEAS
15331,2777,9874734,2026-01-18 00:00:00,sky tripulante,SKY,Empresa,Crédito,0.0,37.0,37.0,0.25,0.0,27.75,0.25,0.75,AEROLINEAS
15332,4086,9869719,2026-01-18 00:00:00,sky tripulante,SKY,Empresa,Crédito,0.0,38.0,38.0,0.25,0.0,28.50,0.25,0.75,AEROLINEAS
15333,3948,9869932,2026-01-18 00:00:00,sky tripulante,SKY,Empresa,Crédito,0.0,49.0,49.0,0.25,0.0,36.75,0.25,0.75,AEROLINEAS
15334,3976,9869885,2026-01-18 00:00:00,sky tripulante,SKY,Empresa,Crédito,13.2,54.0,67.2,0.29,0.0,38.34,0.29,0.71,URBANO PARTNER


In [13093]:
df_matriz =df_dscts.copy()
df_matriz.head()

,Movil,N° Servicio,Fecha,Cliente,Unidad de Negocio,Tipo de Servicio,Tipo de pago,Peaje,Monto a Liquidar,Total Servicio,Comisión,Comisión Aeropuerto,Total para conductor (sin peaje),Comisión Directo,Comisión Conductor,Flota
0,1410,B012-70046,2026-01-12 00:00:00,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
1,794,B012-70048,2026-01-12 00:00:00,KUSI,ORIGEN,TICKET,Crédito,0.0,50.0,50.0,0.279,0.0,36.050,0.279,0.721,AEROPUERTO
2,3815,B012-70045,2026-01-12 00:00:00,KUSI,ORIGEN,TICKET,Crédito,0.0,75.0,75.0,0.279,0.0,54.075,0.279,0.721,AEROPUERTO
3,9656,B012-70044,2026-01-12 00:00:00,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
4,879,B012-70043,2026-01-12 00:00:00,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO


In [13094]:
df_matriz.insert(3, 'Hora','')
df_matriz.head()

,Movil,N° Servicio,Fecha,Hora,Cliente,Unidad de Negocio,Tipo de Servicio,Tipo de pago,Peaje,Monto a Liquidar,Total Servicio,Comisión,Comisión Aeropuerto,Total para conductor (sin peaje),Comisión Directo,Comisión Conductor,Flota
0,1410,B012-70046,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
1,794,B012-70048,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,50.0,50.0,0.279,0.0,36.050,0.279,0.721,AEROPUERTO
2,3815,B012-70045,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,75.0,75.0,0.279,0.0,54.075,0.279,0.721,AEROPUERTO
3,9656,B012-70044,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
4,879,B012-70043,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO


In [13095]:
df_matriz.rename(columns={'Movil':'MOVIL', 'N° Servicio':'Nro_Servicio', 'Unidad de Negocio':'Unidad_Negocio', 
'Tipo de Servicio':'Tipo_Servicio', 'Tipo de pago':'Tipo_Pago', 'Monto a Liquidar':'Monto_Liquidar',
'Total Servicio':'Total_Servicio', 'Comisión':'Comision','Comisión Aeropuerto':'Comision_Aeropuerto',
'Total para conductor  (sin peaje)':'Total_Conductor_Sin_Peaje', 'Comisión Directo':'Comision_Directo', 
'Comisión Conductor':'Comision_Conductor'
}, inplace=True)
df_matriz.head()

,MOVIL,Nro_Servicio,Fecha,Hora,Cliente,Unidad_Negocio,Tipo_Servicio,Tipo_Pago,Peaje,Monto_Liquidar,Total_Servicio,Comision,Comision_Aeropuerto,Total_Conductor_Sin_Peaje,Comision_Directo,Comision_Conductor,Flota
0,1410,B012-70046,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
1,794,B012-70048,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,50.0,50.0,0.279,0.0,36.050,0.279,0.721,AEROPUERTO
2,3815,B012-70045,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,75.0,75.0,0.279,0.0,54.075,0.279,0.721,AEROPUERTO
3,9656,B012-70044,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
4,879,B012-70043,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO


In [13096]:
df_matriz.dropna(subset=["MOVIL"], inplace=True)
df_matriz.head()

,MOVIL,Nro_Servicio,Fecha,Hora,Cliente,Unidad_Negocio,Tipo_Servicio,Tipo_Pago,Peaje,Monto_Liquidar,Total_Servicio,Comision,Comision_Aeropuerto,Total_Conductor_Sin_Peaje,Comision_Directo,Comision_Conductor,Flota
0,1410,B012-70046,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
1,794,B012-70048,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,50.0,50.0,0.279,0.0,36.050,0.279,0.721,AEROPUERTO
2,3815,B012-70045,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,75.0,75.0,0.279,0.0,54.075,0.279,0.721,AEROPUERTO
3,9656,B012-70044,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO
4,879,B012-70043,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO


In [13097]:
df_matriz.tail()

,MOVIL,Nro_Servicio,Fecha,Hora,Cliente,Unidad_Negocio,Tipo_Servicio,Tipo_Pago,Peaje,Monto_Liquidar,Total_Servicio,Comision,Comision_Aeropuerto,Total_Conductor_Sin_Peaje,Comision_Directo,Comision_Conductor,Flota
15330,4049,9869893,2026-01-18 00:00:00,,sky tripulante,SKY,Empresa,Crédito,0.0,54.0,54.0,0.25,0.0,40.50,0.25,0.75,AEROLINEAS
15331,2777,9874734,2026-01-18 00:00:00,,sky tripulante,SKY,Empresa,Crédito,0.0,37.0,37.0,0.25,0.0,27.75,0.25,0.75,AEROLINEAS
15332,4086,9869719,2026-01-18 00:00:00,,sky tripulante,SKY,Empresa,Crédito,0.0,38.0,38.0,0.25,0.0,28.50,0.25,0.75,AEROLINEAS
15333,3948,9869932,2026-01-18 00:00:00,,sky tripulante,SKY,Empresa,Crédito,0.0,49.0,49.0,0.25,0.0,36.75,0.25,0.75,AEROLINEAS
15334,3976,9869885,2026-01-18 00:00:00,,sky tripulante,SKY,Empresa,Crédito,13.2,54.0,67.2,0.29,0.0,38.34,0.29,0.71,URBANO PARTNER


In [13098]:
indices_a_borrar = df_facturas[df_matriz["MOVIL"].astype(str).str.contains("M")].index
indices_a_borrar

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18092\928925832.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indices_a_borrar = df_facturas[df_matriz["MOVIL"].astype(str).str.contains("M")].index


Index([], dtype='int64')

In [13099]:
# Buscamos los índices de las filas que contienen "M"
indices_a_borrar = df_matriz[df_matriz["MOVIL"].astype(str).str.contains("m")].index

# Borramos por esos índices
df_matriz.drop(indices_a_borrar, inplace=True)

In [13100]:
# Buscamos los índices de las filas que contienen "M"
indices_a_borrar = df_matriz[df_matriz["MOVIL"].astype(str).str.contains("M")].index

# Borramos por esos índices
df_matriz.drop(indices_a_borrar, inplace=True)

In [13101]:
df_matriz['anho'] = anho
df_matriz['LIQ'] = liquidacion
df_matriz['id'] = df_matriz['anho'].astype(str) + df_matriz['LIQ'].astype(str) + df_matriz['MOVIL'].astype(float).astype(int).astype(str)
df_matriz.head()

,MOVIL,Nro_Servicio,Fecha,Hora,Cliente,Unidad_Negocio,Tipo_Servicio,Tipo_Pago,Peaje,Monto_Liquidar,Total_Servicio,Comision,Comision_Aeropuerto,Total_Conductor_Sin_Peaje,Comision_Directo,Comision_Conductor,Flota,anho,LIQ,id
0,1410,B012-70046,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO,2026,03,2026031410
1,794,B012-70048,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,50.0,50.0,0.279,0.0,36.050,0.279,0.721,AEROPUERTO,2026,03,202603794
2,3815,B012-70045,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,75.0,75.0,0.279,0.0,54.075,0.279,0.721,AEROPUERTO,2026,03,2026033815
3,9656,B012-70044,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO,2026,03,2026039656
4,879,B012-70043,2026-01-12 00:00:00,,KUSI,ORIGEN,TICKET,Crédito,0.0,60.0,60.0,0.279,0.0,43.260,0.279,0.721,AEROPUERTO,2026,03,202603879


# Lectura Otras Reposiciones

In [13102]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='REPS', 
            header=4
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 1056
Flag1
Flag2

📊 DATOS LISTOS:
    MOVIL  Suma de PAGO                         RAZÓN # VALE/SVS      FECHA  \
0  1549.0           1.0  PROPINA DE SERVICIO #5462425    5462425 2026-01-14   
1  9154.0           5.0  PROPINA DE SERVICIO #5463600    5463600 2026-01-14   
2  2972.0           5.0  PROPINA DE SERVICIO #5459029    5459029 2026-01-12   
3  2416.0           1.0  PROPINA DE SERVICIO #5464160    5464160 2026-01-15   
4  3840.0           1.0  PROPINA DE SERVICIO #5463067    5463067 2026-01-15   

   TARIFA BRUTA SUB DETALLE  MOVIL.1  Suma de PAGO.1         RAZÓN.1  \
0           NaN    Propinas   3517.0            13.2  REGULARIZACIÓN   
1           NaN    Propinas   3891.0            13.2  REGULARIZACIÓN   
2           NaN    Propinas   3303.0    

In [13103]:
df_reps_1 = df_dscts.iloc[:,0:7]
df_reps_1.reset_index(drop=True, inplace=True)
df_reps_1.head()

,MOVIL,Suma de PAGO,RAZÓN,# VALE/SVS,FECHA,TARIFA BRUTA,SUB DETALLE
0,1549.0,1.0,PROPINA DE SERVICIO #5462425,5462425,2026-01-14,NaN,Propinas
1,9154.0,5.0,PROPINA DE SERVICIO #5463600,5463600,2026-01-14,NaN,Propinas
2,2972.0,5.0,PROPINA DE SERVICIO #5459029,5459029,2026-01-12,NaN,Propinas
3,2416.0,1.0,PROPINA DE SERVICIO #5464160,5464160,2026-01-15,NaN,Propinas
4,3840.0,1.0,PROPINA DE SERVICIO #5463067,5463067,2026-01-15,NaN,Propinas


In [13104]:
df_reps_1.rename(columns={'Suma de PAGO':'Suma_de_Pago', 'RAZÓN':'RAZON', '# VALE/SVS':'Nro_Vale_svs', 
'TARIFA BRUTA':'TARIFA_BRUTA', 'SUB DETALLE':'SUB_DETALLE'}, inplace=True)
df_reps_1.dropna(subset=["MOVIL"], inplace=True)
df_reps_1.head()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_svs,FECHA,TARIFA_BRUTA,SUB_DETALLE
0,1549.0,1.0,PROPINA DE SERVICIO #5462425,5462425,2026-01-14,NaN,Propinas
1,9154.0,5.0,PROPINA DE SERVICIO #5463600,5463600,2026-01-14,NaN,Propinas
2,2972.0,5.0,PROPINA DE SERVICIO #5459029,5459029,2026-01-12,NaN,Propinas
3,2416.0,1.0,PROPINA DE SERVICIO #5464160,5464160,2026-01-15,NaN,Propinas
4,3840.0,1.0,PROPINA DE SERVICIO #5463067,5463067,2026-01-15,NaN,Propinas


In [13105]:
df_reps_2 = df_dscts.iloc[:,7:12]
df_reps_2.reset_index(drop=True, inplace=True)
df_reps_2.head()

,MOVIL.1,Suma de PAGO.1,RAZÓN.1,DETALLE,# VALE/SVS.1
0,3517.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,9663308
1,3891.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,B012-67276
2,3303.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,2598344-1
3,1402.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,FW01-00016225
4,1422.0,6.6,REGULARIZACIÓN,Peajes Wally + Nexus,5446441


In [13106]:
df_reps_2.rename(columns={'MOVIL.1':'MOVIL', 'Suma de PAGO.1':'Suma_de_Pago', 'RAZÓN.1':'RAZON',
'DETALLE':'SUB_DETALLE', '# VALE/SVS.1':'Nro_Vale_svs'}, inplace=True)
df_reps_2.dropna(subset=['MOVIL'], inplace=True)
df_reps_2.head()

,MOVIL,Suma_de_Pago,RAZON,SUB_DETALLE,Nro_Vale_svs
0,3517.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,9663308
1,3891.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,B012-67276
2,3303.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,2598344-1
3,1402.0,13.2,REGULARIZACIÓN,Peajes Wally + Nexus,FW01-00016225
4,1422.0,6.6,REGULARIZACIÓN,Peajes Wally + Nexus,5446441


In [13107]:
df_reps_2.tail()

,MOVIL,Suma_de_Pago,RAZON,SUB_DETALLE,Nro_Vale_svs
623,4106.0,6.6,REEMBOLSO DE PEAJE NEXUS,Peajes Wally + Nexus,9841932
624,3976.0,13.2,REEMBOLSO DE PEAJE NEXUS,Peajes Wally + Nexus,9869885
625,2220.0,4.0,REEMBOLSO DE PEAJE NEXUS,Peajes Wally + Nexus,5466677
626,2356.0,4.9,REEMBOLSO DE PEAJE NEXUS,Peajes Wally + Nexus,5457786
627,467.0,7.5,REEMBOLSO DE PEAJE NEXUS,Peajes Wally + Nexus,5464094


In [13108]:
df_reps_3 = df_dscts.iloc[:,12:19]
df_reps_3.reset_index(drop=True, inplace=True)
df_reps_3.head()

,MOVIL.2,Suma de PAGO.2,RAZÓN.2,# VALE/SVS.2,FECHA.1,TARIFA BRUTA.1,SUB DETALLE.1
0,1541.0,2.0,Bono de traslado 5461434,5461434.0,2026-01-13,NaN,Bono de traslado
1,240.0,2.0,Bono de traslado 5458691,5458691.0,2026-01-12,NaN,Bono de traslado
2,2755.0,2.0,Bono de traslado 5459750,5459750.0,2026-01-13,NaN,Bono de traslado
3,412.0,2.0,Bono de traslado 5457740,5457740.0,2026-01-12,NaN,Bono de traslado
4,301.0,2.0,Bono de traslado 5460283,5460283.0,2026-01-13,NaN,Bono de traslado


In [13109]:
df_reps_3.rename(columns={'MOVIL.2':'MOVIL', 'Suma de PAGO.2':'Suma_de_Pago', 'RAZÓN.2':'RAZON',
'# VALE/SVS.2':'Nro_Vale_svs', 'FECHA.1':'FECHA', 'TARIFA BRUTA.1':'TARIFA_BRUTA','SUB DETALLE.1':'SUB_DETALLE'}, inplace=True)
df_reps_3.dropna(subset=['MOVIL'], inplace=True)
df_reps_3.head()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_svs,FECHA,TARIFA_BRUTA,SUB_DETALLE
0,1541.0,2.0,Bono de traslado 5461434,5461434.0,2026-01-13,NaN,Bono de traslado
1,240.0,2.0,Bono de traslado 5458691,5458691.0,2026-01-12,NaN,Bono de traslado
2,2755.0,2.0,Bono de traslado 5459750,5459750.0,2026-01-13,NaN,Bono de traslado
3,412.0,2.0,Bono de traslado 5457740,5457740.0,2026-01-12,NaN,Bono de traslado
4,301.0,2.0,Bono de traslado 5460283,5460283.0,2026-01-13,NaN,Bono de traslado


In [13110]:
df_reps_3.tail()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_svs,FECHA,TARIFA_BRUTA,SUB_DETALLE
659,3719.0,10.0,Bono de traslado 5465554,5465554.0,2026-01-16,NaN,Bono de traslado
660,627.0,10.0,Bono de traslado 5467641,5467641.0,2026-01-17,NaN,Bono de traslado
661,643.0,10.0,Bono de traslado 5469344,5469344.0,2026-01-18,NaN,Bono de traslado
662,9226.0,10.0,Bono de traslado 5468718,5468718.0,2026-01-18,NaN,Bono de traslado
663,596.0,10.0,Bono de traslado 5469606,5469606.0,2026-01-18,NaN,Bono de traslado


In [13111]:
df_reps_total = pd.concat([df_reps_1, df_reps_2, df_reps_3], axis=0, ignore_index=True, sort=False)
df_reps_total.head()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_svs,FECHA,TARIFA_BRUTA,SUB_DETALLE
0,1549.0,1.0,PROPINA DE SERVICIO #5462425,5462425,2026-01-14,NaN,Propinas
1,9154.0,5.0,PROPINA DE SERVICIO #5463600,5463600,2026-01-14,NaN,Propinas
2,2972.0,5.0,PROPINA DE SERVICIO #5459029,5459029,2026-01-12,NaN,Propinas
3,2416.0,1.0,PROPINA DE SERVICIO #5464160,5464160,2026-01-15,NaN,Propinas
4,3840.0,1.0,PROPINA DE SERVICIO #5463067,5463067,2026-01-15,NaN,Propinas


In [13112]:
df_reps_total.dropna(subset=["MOVIL"], inplace=True)
df_reps_total['MOVIL'] = df_reps_total['MOVIL'].astype(float).astype(int).astype(str)

In [13113]:
df_reps_total['anho'] = anho
df_reps_total['LIQ'] = liquidacion
df_reps_total['id'] = df_reps_total['anho'].astype(str) + df_reps_total['LIQ'].astype(str) + df_reps_total['MOVIL'].astype(float).astype(int).astype(str)
df_reps_total.head()

,MOVIL,Suma_de_Pago,RAZON,Nro_Vale_svs,FECHA,TARIFA_BRUTA,SUB_DETALLE,anho,LIQ,id
0,1549,1.0,PROPINA DE SERVICIO #5462425,5462425,2026-01-14,NaN,Propinas,2026,03,2026031549
1,9154,5.0,PROPINA DE SERVICIO #5463600,5463600,2026-01-14,NaN,Propinas,2026,03,2026039154
2,2972,5.0,PROPINA DE SERVICIO #5459029,5459029,2026-01-12,NaN,Propinas,2026,03,2026032972
3,2416,1.0,PROPINA DE SERVICIO #5464160,5464160,2026-01-15,NaN,Propinas,2026,03,2026032416
4,3840,1.0,PROPINA DE SERVICIO #5463067,5463067,2026-01-15,NaN,Propinas,2026,03,2026033840


# Lectura Fondo Individual

In [13114]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='FONDO INDIVIDUAL', 
            header=0
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts.columns =  df_dscts.columns.astype(str)
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 72
Flag1
Flag2

📊 DATOS LISTOS:
   MOVIL                       CONDUCTOR       FLOTA  LIQ#01  LIQ#02  LIQ#03  \
0    735    CACERES PIZANGO LUIS ENRIQUE  AEROPUERTO     200     200     200   
1    950  PAROY MONTERO JAVIER ALEJANDRO  AEROPUERTO     200     200     200   
2   1406    MACEDO MONTES CHARLES DEXTRE      URBANO     200     200     200   
3   1416          QUISPE ALVAREZ RICARDO  AEROPUERTO     200     200     200   
4   1458      LEIVA ROSARIO RONY AGUSTIN  AEROPUERTO     200     200     200   

   LIQ#04  LIQ#05  LIQ#06  LIQ#07  ...  LIQ#42  LIQ#43  LIQ#44  LIQ#45  \
0       0       0       0       0  ...       0       0       0       0   
1       0       0       0       0  ...       0       0       0       0   
2       0       0       0     

In [13115]:
# 1. Identificar las columnas que tienen "LIQ" en su nombre
cols_a_borrar = [col for col in df_dscts.columns if 'LIQ' in str(col)]

# 2. Borrarlas
df_fondo_individual = df_dscts.drop(columns=cols_a_borrar)

print(f"Se eliminaron: {cols_a_borrar}")
df_fondo_individual.head()

Se eliminaron: ['LIQ#01', 'LIQ#02', 'LIQ#03', 'LIQ#04', 'LIQ#05', 'LIQ#06', 'LIQ#07', 'LIQ#08', 'LIQ#09', 'LIQ#10', 'LIQ#11', 'LIQ#12', 'LIQ#13', 'LIQ#14', 'LIQ#15', 'LIQ#16', 'LIQ#17', 'LIQ#18', 'LIQ#19', 'LIQ#20', 'LIQ#21', 'LIQ#22', 'LIQ#23', 'LIQ#24', 'LIQ#25', 'LIQ#26', 'LIQ#27', 'LIQ#28', 'LIQ#29', 'LIQ#30', 'LIQ#31', 'LIQ#32', 'LIQ#33', 'LIQ#34', 'LIQ#35', 'LIQ#36', 'LIQ#37', 'LIQ#38', 'LIQ#39', 'LIQ#40', 'LIQ#41', 'LIQ#42', 'LIQ#43', 'LIQ#44', 'LIQ#45', 'LIQ#46', 'LIQ#47', 'LIQ#48']


,MOVIL,CONDUCTOR,FLOTA,TOTAL ACUMULADO,VALOR CUOTA SEMANAL = S/100.00,150
0,735,CACERES PIZANGO LUIS ENRIQUE,AEROPUERTO,600,NaN,NaN
1,950,PAROY MONTERO JAVIER ALEJANDRO,AEROPUERTO,600,NaN,NaN
2,1406,MACEDO MONTES CHARLES DEXTRE,URBANO,600,NaN,NaN
3,1416,QUISPE ALVAREZ RICARDO,AEROPUERTO,600,NaN,NaN
4,1458,LEIVA ROSARIO RONY AGUSTIN,AEROPUERTO,600,NaN,NaN


In [13116]:
df_fondo_individual = df_fondo_individual.iloc[:,0:4]
df_fondo_individual.reset_index(drop=True, inplace=True)
df_fondo_individual['TOTAL ACUMULADO'] = df_fondo_individual['TOTAL ACUMULADO'].astype(float)
df_fondo_individual.head()

,MOVIL,CONDUCTOR,FLOTA,TOTAL ACUMULADO
0,735,CACERES PIZANGO LUIS ENRIQUE,AEROPUERTO,600.0
1,950,PAROY MONTERO JAVIER ALEJANDRO,AEROPUERTO,600.0
2,1406,MACEDO MONTES CHARLES DEXTRE,URBANO,600.0
3,1416,QUISPE ALVAREZ RICARDO,AEROPUERTO,600.0
4,1458,LEIVA ROSARIO RONY AGUSTIN,AEROPUERTO,600.0


In [13117]:
df_fondo_individual.rename(columns={'TOTAL ACUMULADO':'TOTAL_ACUMULADO'}, inplace=True)
df_fondo_individual.head()

,MOVIL,CONDUCTOR,FLOTA,TOTAL_ACUMULADO
0,735,CACERES PIZANGO LUIS ENRIQUE,AEROPUERTO,600.0
1,950,PAROY MONTERO JAVIER ALEJANDRO,AEROPUERTO,600.0
2,1406,MACEDO MONTES CHARLES DEXTRE,URBANO,600.0
3,1416,QUISPE ALVAREZ RICARDO,AEROPUERTO,600.0
4,1458,LEIVA ROSARIO RONY AGUSTIN,AEROPUERTO,600.0


In [13118]:
df_fondo_individual.dropna(subset=["MOVIL"], inplace=True)
df_fondo_individual[df_fondo_individual.isna()].head()

,MOVIL,CONDUCTOR,FLOTA,TOTAL_ACUMULADO
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [13119]:
df_fondo_individual['anho'] = anho
df_fondo_individual['LIQ'] = liquidacion
df_fondo_individual['id'] = df_fondo_individual['anho'].astype(str) + df_fondo_individual['LIQ'].astype(str) + df_fondo_individual['MOVIL'].astype(float).astype(int).astype(str)
df_fondo_individual.head()

,MOVIL,CONDUCTOR,FLOTA,TOTAL_ACUMULADO,anho,LIQ,id
0,735,CACERES PIZANGO LUIS ENRIQUE,AEROPUERTO,600.0,2026,03,202603735
1,950,PAROY MONTERO JAVIER ALEJANDRO,AEROPUERTO,600.0,2026,03,202603950
2,1406,MACEDO MONTES CHARLES DEXTRE,URBANO,600.0,2026,03,2026031406
3,1416,QUISPE ALVAREZ RICARDO,AEROPUERTO,600.0,2026,03,2026031416
4,1458,LEIVA ROSARIO RONY AGUSTIN,AEROPUERTO,600.0,2026,03,2026031458


In [13120]:
len(df_fondo_individual)

72

# Lectura No pagados

In [13121]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='NO PAGADOS', 
            header=1
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts.columns =  df_dscts.columns.astype(str)
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'N° MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'N° MOVIL' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 815
Flag1
Flag2

📊 DATOS LISTOS:
   N° MOVIL MONTO A EMITIR PROACCION  MONTO A EMITIR TRANSPORTES  \
0    9107.0                        0                    114.7500   
1     756.0                        0                    390.4059   
2    2647.0                        0                      0.0000   
3    2277.0                    230.8                     37.5000   
4    9048.0                     65.8                     12.6000   

  ADELANTO Y/O REINGRESOS  MONTO A PAGAR PROACCION  \
0                     NaN               -2511.6000   
1                     NaN               -2257.6291   
2                     NaN               -2020.2000   
3                     NaN               -1954.6000   
4                     NaN               -1925.9500   

In [13122]:
df_no_pagados_1 = df_dscts.iloc[:,0:6]
df_no_pagados_1.drop(columns=['ADELANTO Y/O REINGRESOS'], inplace=True)
df_no_pagados_1.rename(columns={'N° MOVIL': 'MOVIL','MONTO A EMITIR PROACCION':'MONTO_EMITIR_PROACCION',
'MONTO A EMITIR TRANSPORTES':'MONTO_EMITIR_TRANSPORTES','MONTO A PAGAR PROACCION':'MONTO_PAGAR_PROACCION',
'DETALLE (POSITIVO/NEGATIVO)':'DETALLE_PROACCION'}, inplace=True)
df_no_pagados_1.reset_index(drop=True, inplace=True)
df_no_pagados_1.dropna(subset=['MOVIL'], inplace=True)
df_no_pagados_1.head()

,MOVIL,MONTO_EMITIR_PROACCION,MONTO_EMITIR_TRANSPORTES,MONTO_PAGAR_PROACCION,DETALLE_PROACCION
0,9107.0,0,114.7500,-2511.6000,NEGATIVO
1,756.0,0,390.4059,-2257.6291,NEGATIVO
2,2647.0,0,0.0000,-2020.2000,NEGATIVO
3,2277.0,230.8,37.5000,-1954.6000,NEGATIVO
4,9048.0,65.8,12.6000,-1925.9500,NEGATIVO


In [13123]:
df_no_pagados_2 = df_dscts.iloc[:,6:10]
df_no_pagados_2.drop(columns=['ADELANTO Y/O REINGRESOS.1'], inplace=True)
df_no_pagados_2.rename(columns={'DETALLE (POSITIVO/NEGATIVO).1': 'DETALLE_TRANSPORTE','N° MOVIL.1': 'MOVIL',
'MONTO A PAGAR TRANSPORTES':'MONTO_PAGAR_TRANSPORTES'}, inplace=True)
df_no_pagados_2.reset_index(drop=True, inplace=True)
df_no_pagados_2.dropna(subset=['MOVIL'], inplace=True)
df_no_pagados_2.head()

,MOVIL,MONTO_PAGAR_TRANSPORTES,DETALLE_TRANSPORTE
0,1524.0,-192.5120,NEGATIVO
1,923.0,-81.5806,NEGATIVO
2,3956.0,-78.6250,NEGATIVO
3,807.0,-78.5100,NEGATIVO
4,217.0,-76.7334,NEGATIVO


In [13124]:
df_no_pagados_total = pd.concat([df_no_pagados_1, df_no_pagados_2], axis=0, ignore_index=True, sort=False)
df_no_pagados_total.head()

,MOVIL,MONTO_EMITIR_PROACCION,MONTO_EMITIR_TRANSPORTES,MONTO_PAGAR_PROACCION,DETALLE_PROACCION,MONTO_PAGAR_TRANSPORTES,DETALLE_TRANSPORTE
0,9107.0,0,114.7500,-2511.6000,NEGATIVO,NaN,NaN
1,756.0,0,390.4059,-2257.6291,NEGATIVO,NaN,NaN
2,2647.0,0,0.0000,-2020.2000,NEGATIVO,NaN,NaN
3,2277.0,230.8,37.5000,-1954.6000,NEGATIVO,NaN,NaN
4,9048.0,65.8,12.6000,-1925.9500,NEGATIVO,NaN,NaN


In [13125]:
df_no_pagados_total['anho'] = anho
df_no_pagados_total['LIQ'] = liquidacion
df_no_pagados_total['id'] = df_no_pagados_total['anho'].astype(str) + df_no_pagados_total['LIQ'].astype(str) + df_no_pagados_total['MOVIL'].astype(float).astype(int).astype(str)
df_no_pagados_total.head()

,MOVIL,MONTO_EMITIR_PROACCION,MONTO_EMITIR_TRANSPORTES,MONTO_PAGAR_PROACCION,DETALLE_PROACCION,MONTO_PAGAR_TRANSPORTES,DETALLE_TRANSPORTE,anho,LIQ,id
0,9107.0,0,114.7500,-2511.6000,NEGATIVO,NaN,NaN,2026,03,2026039107
1,756.0,0,390.4059,-2257.6291,NEGATIVO,NaN,NaN,2026,03,202603756
2,2647.0,0,0.0000,-2020.2000,NEGATIVO,NaN,NaN,2026,03,2026032647
3,2277.0,230.8,37.5000,-1954.6000,NEGATIVO,NaN,NaN,2026,03,2026032277
4,9048.0,65.8,12.6000,-1925.9500,NEGATIVO,NaN,NaN,2026,03,2026039048


# Lectura Liquidaciones

In [13126]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='LIQUIDACION', 
            header=4
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts.columns =  df_dscts.columns.astype(str)
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MOVIL' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 2145
Flag1
Flag2

📊 DATOS LISTOS:
               PLAZAS  MOVIL           CONDUCTOR          PROPETARIO  \
0  MOVIL CONTINGENCIA    1.0  MOVIL CONTINGENCIA  MOVIL CONTINGENCIA   
1  MOVIL CONTINGENCIA    5.0  MOVIL CONTINGENCIA  MOVIL CONTINGENCIA   
2  MOVIL CONTINGENCIA    6.0  MOVIL CONTINGENCIA  MOVIL CONTINGENCIA   
3  MOVIL CONTINGENCIA    7.0  MOVIL CONTINGENCIA  MOVIL CONTINGENCIA   
4  MOVIL CONTINGENCIA   12.0  MOVIL CONTINGENCIA  MOVIL CONTINGENCIA   

   PRODUCCION BRUTA CORPORATIVOS  COMISION DIRECTO CORPORATIVOS  \
0                           0.00                           0.00   
1                        3431.90                        3431.90   
2                        2119.42                        2119.42   
3                        3033.

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [13127]:
df_liquidacion = df_dscts.copy()
df_liquidacion.head()

,PLAZAS,MOVIL,CONDUCTOR,PROPETARIO,PRODUCCION BRUTA CORPORATIVOS,COMISION DIRECTO CORPORATIVOS,PRODUCCION NETA CONDUCTOR CORPORATIVOS,PRODUCCION BRUTA NEXUS,COMISION TD NEXUS,PRODUCCION NETA CONDUCTOR NEXUS,...,DOCUMENTO,BANCO,N.º CUENTA,MONTO A EMITIR PROACCION,MONTO A EMITIR TRANSPORTES,MONTO A EMITIR PROACCION (LIQ ANTERIOR),MONTO A EMITIR TRANSPORTES (LIQ ANTERIOR),MONTO A EMITIR PROACCION TOTAL,MONTO A EMITIR TRANSPORTES TOTAL,SERVICIO CONTABLE
0,MOVIL CONTINGENCIA,1.0,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.00,0.00,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
1,MOVIL CONTINGENCIA,5.0,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,3431.90,3431.90,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
2,MOVIL CONTINGENCIA,6.0,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,2119.42,2119.42,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
3,MOVIL CONTINGENCIA,7.0,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,3033.00,3033.00,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
4,MOVIL CONTINGENCIA,12.0,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.00,0.00,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO


In [13128]:
df_liquidacion.columns

Index(['PLAZAS', 'MOVIL', 'CONDUCTOR', 'PROPETARIO',
       'PRODUCCION BRUTA CORPORATIVOS', 'COMISION DIRECTO CORPORATIVOS',
       'PRODUCCION NETA CONDUCTOR CORPORATIVOS', 'PRODUCCION BRUTA NEXUS',
       'COMISION TD NEXUS', 'PRODUCCION NETA CONDUCTOR NEXUS',
       'PRODUCCION BRUTA NEXUS EFECTIVO', 'COMISION TD NEXUS EFECTIVO',
       'TOTAL PROD BRUTA CRED Y EFECT CONDUCTOR',
       'TOTAL PROD BRUTA CREDITO CONDUCTOR',
       'TOTAL PROD BRUTA CREDITO CONDUCTOR.1', 'TOTAL COMISION TD CREDITO',
       'PRODUCCION NETA CONDUCTOR CREDITO', '2026-01-12 00:00:00',
       '2026-01-13 00:00:00', '2026-01-14 00:00:00', '2026-01-15 00:00:00',
       '2026-01-16 00:00:00', '2026-01-17 00:00:00', '2026-01-18 00:00:00',
       'MEMBRESIA SEMANAL', 'ASITENCIA (A)', 'URBANO/ LATAM/VAN', 'DESACT',
       'NUEVO (-)', 'PERMISO ESPECIAL (PE)', 'CUPON DE PERMISO (CP)',
       'MEMBRESIA FINAL', 'LIQUIDACION ANTERIOR (PARA PAGO)', 'PEAJE NEXUS',
       'BONO TRASLADO', 'PEAJE NEXUS + WALLY', 'OTR

In [13129]:
cols_a_borrar = [c for c in df_liquidacion.columns if '00:00:00' in str(c) or '-' in str(c)]

df_liquidacion = df_liquidacion.drop(columns=cols_a_borrar)

In [13130]:
df_liquidacion.columns

Index(['PLAZAS', 'MOVIL', 'CONDUCTOR', 'PROPETARIO',
       'PRODUCCION BRUTA CORPORATIVOS', 'COMISION DIRECTO CORPORATIVOS',
       'PRODUCCION NETA CONDUCTOR CORPORATIVOS', 'PRODUCCION BRUTA NEXUS',
       'COMISION TD NEXUS', 'PRODUCCION NETA CONDUCTOR NEXUS',
       'PRODUCCION BRUTA NEXUS EFECTIVO', 'COMISION TD NEXUS EFECTIVO',
       'TOTAL PROD BRUTA CRED Y EFECT CONDUCTOR',
       'TOTAL PROD BRUTA CREDITO CONDUCTOR',
       'TOTAL PROD BRUTA CREDITO CONDUCTOR.1', 'TOTAL COMISION TD CREDITO',
       'PRODUCCION NETA CONDUCTOR CREDITO', 'MEMBRESIA SEMANAL',
       'ASITENCIA (A)', 'URBANO/ LATAM/VAN', 'DESACT', 'PERMISO ESPECIAL (PE)',
       'CUPON DE PERMISO (CP)', 'MEMBRESIA FINAL',
       'LIQUIDACION ANTERIOR (PARA PAGO)', 'PEAJE NEXUS', 'BONO TRASLADO',
       'PEAJE NEXUS + WALLY', 'OTRAS REP.', 'TOTAL REPOSICIONES',
       'LIQUIDACION ANTERIOR PROACCION', 'LIQUIDACION ANTERIOR TRANSPORTES',
       'COMISIÓN URBANO', 'FONDO INDIVIDUAL', 'DSCTO ACTIVOS',
       'DSCTO RE

In [13131]:
df_liquidacion.rename(columns={
    'PLAZAS': 'Plazas',
    'MOVIL': 'Movil',
    'CONDUCTOR': 'Conductor',
    'PROPETARIO': 'Propetario',
    'PRODUCCION BRUTA CORPORATIVOS': 'Produccion_Bruta_Corporativos',
    'COMISION DIRECTO CORPORATIVOS': 'Comision_Directo_Corporativos',
    'PRODUCCION NETA CONDUCTOR CORPORATIVOS': 'Produccion_Neta_Conductor_Corporativos',
    'PRODUCCION BRUTA NEXUS': 'Produccion_Bruta_Nexus',
    'COMISION TD NEXUS': 'Comision_Td_Nexus',
    'PRODUCCION NETA CONDUCTOR NEXUS': 'Produccion_Neta_Conductor_Nexus',
    'PRODUCCION BRUTA NEXUS EFECTIVO': 'Produccion_Bruta_Nexus_Efectivo',
    'COMISION TD NEXUS EFECTIVO': 'Comision_Td_Nexus_Efectivo',
    'TOTAL PROD BRUTA CRED Y EFECT CONDUCTOR': 'Total_Prod_Bruta_Cred_Y_Efect_Conductor',
    'TOTAL PROD BRUTA CREDITO CONDUCTOR': 'Total_Prod_Bruta_Credito_Conductor',
    'TOTAL PROD BRUTA CREDITO CONDUCTOR.1': 'Total_Prod_Bruta_Credito_Conductor_1',
    'TOTAL COMISION TD CREDITO': 'Total_Comision_Td_Credito',
    'PRODUCCION NETA CONDUCTOR CREDITO': 'Produccion_Neta_Conductor_Credito',
    
    # Fechas limpias
    '2025-12-01 00:00:00': '2025-12-01',
    '2025-12-02 00:00:00': '2025-12-02',
    '2025-12-03 00:00:00': '2025-12-03',
    '2025-12-04 00:00:00': '2025-12-04',
    '2025-12-05 00:00:00': '2025-12-05',
    '2025-12-06 00:00:00': '2025-12-06',
    '2025-12-07 00:00:00': '2025-12-07',
    
    'MEMBRESIA SEMANAL': 'Membresia_Semanal',
    'ASITENCIA (A)': 'Asitencia',
    'URBANO/ LATAM/VAN': 'Urbano_Latam_Van',
    'DESACT': 'Desact',
    'NUEVO (-)': 'Nuevo',
    'PERMISO ESPECIAL (PE)': 'Permiso_Especial',
    'CUPON DE PERMISO (CP)': 'Cupon_De_Permiso',
    'MEMBRESIA FINAL': 'Membresia_Final',
    'LIQUIDACION ANTERIOR (PARA PAGO)': 'Liquidacion_Anterior',
    'PEAJE NEXUS': 'Peaje_Nexus',
    'BONO TRASLADO': 'Bono_Traslado',
    'PEAJE NEXUS + WALLY': 'Peaje_Nexus_Wally',
    'OTRAS REP.': 'Otras_Rep',
    'TOTAL REPOSICIONES': 'Total_Reposiciones',
    'LIQUIDACION ANTERIOR PROACCION': 'Liquidacion_Anterior_Proaccion',
    'LIQUIDACION ANTERIOR TRANSPORTES': 'Liquidacion_Anterior_Transportes',
    'COMISIÓN URBANO': 'Comision_Urbano',
    'FONDO INDIVIDUAL': 'Fondo_Individual',
    'DSCTO ACTIVOS': 'Dscto_Activos',
    'DSCTO REGULARIZACIÓN': 'Dscto_Regularizacion',
    'DSCTO. MENBRESÍA PEX': 'Dscto_Menbresia_Pex',
    'TOTAL DESCUENTOS': 'Total_Descuentos',
    'PAGO A CONDUCTOR': 'Pago_A_Conductor',
    'RXH': 'Rxh',
    'MONTO A PAGAR': 'Monto_A_Pagar',
    'ESTADO': 'Estado',
    'TIPO DE COMPROBANTE': 'Tipo_De_Comprobante',
    'FACT ELECTRONICA - PROACCIÓN EMPRESARIAL\n': 'Fact_Electronica_Proaccion_Empresarial',
    'FACT ELECTRONICA - TRANSPORTES LIMEÑOS\n': 'Fact_Electronica_Transportes_Limenos',
    'DOCUMENTO': 'Documento',
    'BANCO': 'Banco',
    'N.º CUENTA': 'N_Cuenta',
    'MONTO A EMITIR PROACCION': 'Monto_A_Emitir_Proaccion',
    'MONTO A EMITIR TRANSPORTES': 'Monto_A_Emitir_Transportes',
    
    # Aquí mantuve "Liq_Anterior" sin paréntesis para que no se duplique con las columnas de arriba
    'MONTO A EMITIR PROACCION (LIQ ANTERIOR)': 'Monto_A_Emitir_Proaccion_Liq_Anterior',
    'MONTO A EMITIR TRANSPORTES (LIQ ANTERIOR)': 'Monto_A_Emitir_Transportes_Liq_Anterior',
    
    'MONTO A EMITIR PROACCION TOTAL': 'Monto_A_Emitir_Proaccion_Total',
    'MONTO A EMITIR TRANSPORTES TOTAL': 'Monto_A_Emitir_Transportes_Total',
    'SERVICIO CONTABLE': 'Servicio_Contable'
}, inplace=True)
df_liquidacion.columns

Index(['Plazas', 'Movil', 'Conductor', 'Propetario',
       'Produccion_Bruta_Corporativos', 'Comision_Directo_Corporativos',
       'Produccion_Neta_Conductor_Corporativos', 'Produccion_Bruta_Nexus',
       'Comision_Td_Nexus', 'Produccion_Neta_Conductor_Nexus',
       'Produccion_Bruta_Nexus_Efectivo', 'Comision_Td_Nexus_Efectivo',
       'Total_Prod_Bruta_Cred_Y_Efect_Conductor',
       'Total_Prod_Bruta_Credito_Conductor',
       'Total_Prod_Bruta_Credito_Conductor_1', 'Total_Comision_Td_Credito',
       'Produccion_Neta_Conductor_Credito', 'Membresia_Semanal', 'Asitencia',
       'Urbano_Latam_Van', 'Desact', 'Permiso_Especial', 'Cupon_De_Permiso',
       'Membresia_Final', 'Liquidacion_Anterior', 'Peaje_Nexus',
       'Bono_Traslado', 'Peaje_Nexus_Wally', 'Otras_Rep', 'Total_Reposiciones',
       'Liquidacion_Anterior_Proaccion', 'Liquidacion_Anterior_Transportes',
       'Comision_Urbano', 'Fondo_Individual', 'Dscto_Activos',
       'Dscto_Regularizacion', 'Dscto_Menbresia_Pex',

In [13132]:
df_liquidacion.dropna(subset=["Movil"], inplace=True)
df_liquidacion['Movil'] = df_liquidacion['Movil'].astype(float).astype(int).astype(str)
df_liquidacion.head()

,Plazas,Movil,Conductor,Propetario,Produccion_Bruta_Corporativos,Comision_Directo_Corporativos,Produccion_Neta_Conductor_Corporativos,Produccion_Bruta_Nexus,Comision_Td_Nexus,Produccion_Neta_Conductor_Nexus,...,Documento,Banco,N_Cuenta,Monto_A_Emitir_Proaccion,Monto_A_Emitir_Transportes,Monto_A_Emitir_Proaccion_Liq_Anterior,Monto_A_Emitir_Transportes_Liq_Anterior,Monto_A_Emitir_Proaccion_Total,Monto_A_Emitir_Transportes_Total,Servicio_Contable
0,MOVIL CONTINGENCIA,1,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.00,0.00,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
1,MOVIL CONTINGENCIA,5,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,3431.90,3431.90,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
2,MOVIL CONTINGENCIA,6,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,2119.42,2119.42,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
3,MOVIL CONTINGENCIA,7,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,3033.00,3033.00,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO
4,MOVIL CONTINGENCIA,12,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.00,0.00,0.0,0.0,0.0,0.0,...,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.0,0.0,0.0,0.0,0.0,0.0,NO


In [13133]:
df_liquidacion['anho'] = anho
df_liquidacion['LIQ'] = liquidacion
df_liquidacion['id'] = df_liquidacion['anho'].astype(str) + df_liquidacion['LIQ'].astype(str) + df_liquidacion['Movil'].astype(float).astype(int).astype(str)
df_liquidacion.head()

,Plazas,Movil,Conductor,Propetario,Produccion_Bruta_Corporativos,Comision_Directo_Corporativos,Produccion_Neta_Conductor_Corporativos,Produccion_Bruta_Nexus,Comision_Td_Nexus,Produccion_Neta_Conductor_Nexus,...,Monto_A_Emitir_Proaccion,Monto_A_Emitir_Transportes,Monto_A_Emitir_Proaccion_Liq_Anterior,Monto_A_Emitir_Transportes_Liq_Anterior,Monto_A_Emitir_Proaccion_Total,Monto_A_Emitir_Transportes_Total,Servicio_Contable,anho,LIQ,id
0,MOVIL CONTINGENCIA,1,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,2026,03,2026031
1,MOVIL CONTINGENCIA,5,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,3431.90,3431.90,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,2026,03,2026035
2,MOVIL CONTINGENCIA,6,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,2119.42,2119.42,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,2026,03,2026036
3,MOVIL CONTINGENCIA,7,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,3033.00,3033.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,2026,03,2026037
4,MOVIL CONTINGENCIA,12,MOVIL CONTINGENCIA,MOVIL CONTINGENCIA,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,2026,03,20260312


In [13134]:
len(df_liquidacion.columns)

56

# Lectura de Cupones

In [13135]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='CONTROL DE CUPONES', 
            header=3
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts.columns =  df_dscts.columns.astype(str)
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MOVIL' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 325
Flag1
Flag2

📊 DATOS LISTOS:
  PLACA  MOVIL                       CONDUCTOR N°03  SALDO A LA LIQ N°03  \
0     0   2190  BLANCO SANDOVAL FREDDY ANTONIO    0                    0   
1     0    373     PIZARRO VEGA EDDER YANCARLO    0                    2   
2     0    380               PEREZ DIAS ANTERO    0                    2   
3     0    412  UNZUETA FAJARDO RENÁN STENDHAL    0                    2   
4     0    423              NERLI DIAZ MUNDACA    0                    1   

   N°03.1  NUEVO SALDO  
0       0            0  
1      -1            1  
2       0            2  
3      -2            0  
4      -1            0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column          

In [13136]:
df_cupones = df_dscts.copy()
df_cupones.head()

,PLACA,MOVIL,CONDUCTOR,N°03,SALDO A LA LIQ N°03,N°03.1,NUEVO SALDO
0,0,2190,BLANCO SANDOVAL FREDDY ANTONIO,0,0,0,0
1,0,373,PIZARRO VEGA EDDER YANCARLO,0,2,-1,1
2,0,380,PEREZ DIAS ANTERO,0,2,0,2
3,0,412,UNZUETA FAJARDO RENÁN STENDHAL,0,2,-2,0
4,0,423,NERLI DIAZ MUNDACA,0,1,-1,0


In [13137]:
df_cupones.tail()

,PLACA,MOVIL,CONDUCTOR,N°03,SALDO A LA LIQ N°03,N°03.1,NUEVO SALDO
320,NaN,4128,More Atoche Enrique,PERMISO,0,0,0
321,NaN,4131,BAUTISTA ARAJU CARLOS RAUL,2,2,0,2
322,NaN,4142,Villanueva Tapia Oscar Andres,PERMISO,0,0,0
323,NaN,4144,Raga Briceño Anthony Enrique,2,2,0,2
324,NaN,4145,Zuñiga Escate Carlos ever,2,2,0,2


In [13138]:
df_cupones.drop('PLACA', axis=1, inplace=True)
df_cupones.drop('NUEVO SALDO', axis=1, inplace=True)
df_cupones.drop('CONDUCTOR', axis=1, inplace=True)
df_cupones.head()

,MOVIL,N°03,SALDO A LA LIQ N°03,N°03.1
0,2190,0,0,0
1,373,0,2,-1
2,380,0,2,0
3,412,0,2,-2
4,423,0,1,-1


In [13139]:
df_cupones.rename(columns={df_cupones.columns[1]:'GANADOS'}, inplace=True)
df_cupones.rename(columns={df_cupones.columns[2]:'SALDO INICIAL'}, inplace=True)
df_cupones.rename(columns={df_cupones.columns[3]:'USADOS'}, inplace=True)
df_cupones.head()

,MOVIL,GANADOS,SALDO INICIAL,USADOS
0,2190,0,0,0
1,373,0,2,-1
2,380,0,2,0
3,412,0,2,-2
4,423,0,1,-1


In [13140]:
df_cupones['GANADOS'].unique()

array([0, 'PERMISO', 1, 2], dtype=object)

In [13141]:
df_cupones['GANADOS'].unique()
df_cupones['GANADOS'].replace({'PERMISO': ''}, regex=True, inplace=True)
df_cupones['GANADOS'].replace({'DESACTIVADO': ''}, regex=True, inplace=True)
df_cupones['GANADOS'].replace({'': 0}, regex=True, inplace=True)
df_cupones['GANADOS'].unique()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18092\369465737.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cupones['GANADOS'].replace({'PERMISO': ''}, regex=True, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18092\369465737.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

array([0, 1, 2])

In [13142]:
df_cupones['SALDO FINAL'] = df_cupones['GANADOS'].astype(float) + df_cupones['SALDO INICIAL'].astype(float)
df_cupones.head()

,MOVIL,GANADOS,SALDO INICIAL,USADOS,SALDO FINAL
0,2190,0,0,0,0.0
1,373,0,2,-1,2.0
2,380,0,2,0,2.0
3,412,0,2,-2,2.0
4,423,0,1,-1,1.0


In [13143]:
df_cupones['anho'] = anho
df_cupones['LIQ'] = liquidacion
df_cupones['id'] = df_cupones['anho'].astype(str) + df_cupones['LIQ'].astype(str) + df_cupones['MOVIL'].astype(float).astype(int).astype(str)
df_cupones.head()

,MOVIL,GANADOS,SALDO INICIAL,USADOS,SALDO FINAL,anho,LIQ,id
0,2190,0,0,0,0.0,2026,03,2026032190
1,373,0,2,-1,2.0,2026,03,202603373
2,380,0,2,0,2.0,2026,03,202603380
3,412,0,2,-2,2.0,2026,03,202603412
4,423,0,1,-1,1.0,2026,03,202603423


# Lectura Tipo Vehículo

In [13144]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='BD', 
            header=0
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts.columns =  df_dscts.columns.astype(str)
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MOVIL' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 3280
Flag1
Flag2

📊 DATOS LISTOS:
   MOVIL VEHICULO       MODALIDAD  ESTADO   PLACA  \
0  200.0      200  CERO MATRICULA  ACTIVO  H2H504   
1  201.0      201  CERO MATRICULA  ACTIVO  AU0432   
2  202.0      202  CERO MATRICULA  ACTIVO  ATT007   
3  203.0      203  CERO MATRICULA  ACTIVO  BZB105   
4  204.0      204  CERO MATRICULA  ACTIVO  AWS428   

                          CONDUCTOR                          PROPIETARIO  \
0              CABRERA HECTOR JESUS                 CABRERA HECTOR JESUS   
1    ESPINOZA CARHUATOCTO JOSE LUIS       ESPINOZA CARHUATOCTO JOSE LUIS   
2  REGINALDO CLAROS Felipe Alberto   REGINALDO GUERRERO RACHEL MADELEINE   
3      GALARZA SUAREZ CESAR ALBERTO         GALARZA SUAREZ CESAR ALBERTO   
4         ACOSTA AGUILAR LUIS AN

In [13145]:
df_dscts.tail()

,MOVIL,VEHICULO,MODALIDAD,ESTADO,PLACA,CONDUCTOR,PROPIETARIO,N° DOCUMENTO,BANCO,N° DE CUENTA DEL PROPIETARIO,TIPO DE DOC.,RUC,PLAZA,FECHA DE INICIO,MEMBRESIA,TIPO DE VEHÍCULO
3275,4144.0,4144,CERO MATRICULA,ACTIVO,Bct424,Raga Briceño Anthony Enrique,Raga Briceño Anthony Enrique,008000185,BCP,19104065210072,CE,NaN,NaN,NaN,NaN,NaN
3276,4145.0,4145,CERO MATRICULA,ACTIVO,Csh416,Zuñiga Escate Carlos ever,Zuñiga Escate Carlos Ever,76145420,INTERBANK,8983465952656,DNI,NaN,NaN,NaN,NaN,NaN
3277,4128.0,4128,CERO MATRICULA,ACTIVO,CNB566,More Atoche Enrique,More Atoche Enrique,02892693,BCP,19197840180062,DNI,NaN,NaN,NaN,NaN,NaN
3278,3797.0,3797,CERO MATRICULA,ACTIVO,CJQ369,Santamaria Martínez Oscar Alexander Santamaria...,Santamaria Martínez Oscar Alexander,001661183,BCP,19273340473084,CARNET DE EXTRANJERÍA,NaN,NaN,NaN,NaN,NaN
3279,4131.0,4131,CERO MATRICULA,ACTIVO,Cvr242,BAUTISTA ARAJU CARLOS RAUL,BAUTISTA ARAJU CARLOS RAUL,45605050,BCP,1912703750005,DNI,NaN,NaN,NaN,NaN,NaN


In [13146]:
df_dscts.iloc[3184:3190].head()

,MOVIL,VEHICULO,MODALIDAD,ESTADO,PLACA,CONDUCTOR,PROPIETARIO,N° DOCUMENTO,BANCO,N° DE CUENTA DEL PROPIETARIO,TIPO DE DOC.,RUC,PLAZA,FECHA DE INICIO,MEMBRESIA,TIPO DE VEHÍCULO
3184,4079.0,4079,CERO MATRICULA,ACTIVO,BDP371,Quiquia Machacuay Luís Antonio,Quiquia Machacuay Luís Antonio,72754314,BCP,19198713202003,DNI,10727543142,URBANO PARTNER,2025-12-01 00:00:00,0.0,0
3185,4080.0,4080,CERO MATRICULA,ACTIVO,CUL575,Olivera Rodriguez Alejandro Dennis,Olivera Rodriguez Alejandro Dennis,41288394,BCP,19100725449075,DNI,10412883948,URBANO PARTNER,2025-12-01 00:00:00,0.0,0
3186,4081.0,4081,CERO MATRICULA,ACTIVO,BLK000,Nuñez Alarcon Emerita,Nuñez Alarcon Emerita,44642648,BBVA,001101220200733359,DNI,10446426481,URBANO PARTNER,2025-12-01 00:00:00,0.0,0
3187,4082.0,4082,CERO MATRICULA,ACTIVO,CSP105,Sanchez Zamora Juan Diego,Sanchez Zamora Juan Diego,47138582,BCP,5153621003007,DNI,10471385820,URBANO PARTNER,2025-12-04 00:00:00,0.0,0
3188,4067.0,4067,CERO MATRICULA,ACTIVO,CST559,Avendaño Albites Jhon Agustin,JHON AGUSTIN AVENDAÑO ALBITES,44923120,INTERBANK,0563128766387,DNI,10449231207,MEMBRESÍA NOCTURNA - APTO,2025-11-18 00:00:00,315.0,LITE


In [13147]:
# --- LÓGICA DE CORTE CORREGIDA ---
mask_fila_vacia_total = df_dscts.isna().all(axis=1)

if mask_fila_vacia_total.any():
    # Eliminamos el "- 1". idxmax ya nos da el índice exacto de la primera fila nula.
    primer_indice_vacio = mask_fila_vacia_total.idxmax()
    
    # Al usar iloc[:3187], Pandas tomará hasta la 3186 y se detendrá justo antes del nulo.
    df_dscts = df_dscts.iloc[:primer_indice_vacio]
    print(f"✂️ Corte realizado con éxito en el índice: {primer_indice_vacio}")

# OPCIONAL: Para limpiar cualquier residuo que no sea "Fila Totalmente Vacía"
df_dscts = df_dscts.dropna(how='all')

✂️ Corte realizado con éxito en el índice: 3242


In [13148]:
df_tipo_vehiculo = df_dscts.copy()
df_tipo_vehiculo.head()

,MOVIL,VEHICULO,MODALIDAD,ESTADO,PLACA,CONDUCTOR,PROPIETARIO,N° DOCUMENTO,BANCO,N° DE CUENTA DEL PROPIETARIO,TIPO DE DOC.,RUC,PLAZA,FECHA DE INICIO,MEMBRESIA,TIPO DE VEHÍCULO
0,200.0,200,CERO MATRICULA,ACTIVO,H2H504,CABRERA HECTOR JESUS,CABRERA HECTOR JESUS,007292631,BCP,19497908602075,CE,15609782843,AEROPUERTO,2022-10-14 00:00:00,240.0,XL
1,201.0,201,CERO MATRICULA,ACTIVO,AU0432,ESPINOZA CARHUATOCTO JOSE LUIS,ESPINOZA CARHUATOCTO JOSE LUIS,46896567,BBVA,0011-0814-0211448631,DNI,10468965670,URBANO,NaN,0.0,COMFORT
2,202.0,202,CERO MATRICULA,ACTIVO,ATT007,REGINALDO CLAROS Felipe Alberto,REGINALDO GUERRERO RACHEL MADELEINE,75428748,INTERBANK,0413255172139,DNI,10754287484,AEROLINEAS,2024-04-11 00:00:00,0.0,COMFORT
3,203.0,203,CERO MATRICULA,ACTIVO,BZB105,GALARZA SUAREZ CESAR ALBERTO,GALARZA SUAREZ CESAR ALBERTO,09821702,BCP,191-99089116-0-14,DNI,10098217024,AEROPUERTO,2022-08-19 00:00:00,265.0,LITE
4,204.0,204,CERO MATRICULA,ACTIVO,AWS428,ACOSTA AGUILAR LUIS ANGEL,ACOSTA AGUILAR LUIS ANGEL,48298086,BCP,19105751282074,DNI,10482980860,AEROPUERTO,2022-08-19 00:00:00,235.0,COMFORT XL


In [13149]:
if 'SEGMENTACIÓN' in df_tipo_vehiculo.columns:
    df_tipo_vehiculo['TIPO DE VEHÍCULO'] = 'Sin Definir'

In [13150]:
df_tipo_vehiculo.head()

,MOVIL,VEHICULO,MODALIDAD,ESTADO,PLACA,CONDUCTOR,PROPIETARIO,N° DOCUMENTO,BANCO,N° DE CUENTA DEL PROPIETARIO,TIPO DE DOC.,RUC,PLAZA,FECHA DE INICIO,MEMBRESIA,TIPO DE VEHÍCULO
0,200.0,200,CERO MATRICULA,ACTIVO,H2H504,CABRERA HECTOR JESUS,CABRERA HECTOR JESUS,007292631,BCP,19497908602075,CE,15609782843,AEROPUERTO,2022-10-14 00:00:00,240.0,XL
1,201.0,201,CERO MATRICULA,ACTIVO,AU0432,ESPINOZA CARHUATOCTO JOSE LUIS,ESPINOZA CARHUATOCTO JOSE LUIS,46896567,BBVA,0011-0814-0211448631,DNI,10468965670,URBANO,NaN,0.0,COMFORT
2,202.0,202,CERO MATRICULA,ACTIVO,ATT007,REGINALDO CLAROS Felipe Alberto,REGINALDO GUERRERO RACHEL MADELEINE,75428748,INTERBANK,0413255172139,DNI,10754287484,AEROLINEAS,2024-04-11 00:00:00,0.0,COMFORT
3,203.0,203,CERO MATRICULA,ACTIVO,BZB105,GALARZA SUAREZ CESAR ALBERTO,GALARZA SUAREZ CESAR ALBERTO,09821702,BCP,191-99089116-0-14,DNI,10098217024,AEROPUERTO,2022-08-19 00:00:00,265.0,LITE
4,204.0,204,CERO MATRICULA,ACTIVO,AWS428,ACOSTA AGUILAR LUIS ANGEL,ACOSTA AGUILAR LUIS ANGEL,48298086,BCP,19105751282074,DNI,10482980860,AEROPUERTO,2022-08-19 00:00:00,235.0,COMFORT XL


In [13151]:
columnas_deseadas = ['MOVIL', 'PLACA', 'TIPO DE VEHÍCULO']
df_tipo_vehiculo = df_tipo_vehiculo[columnas_deseadas].copy()
df_tipo_vehiculo.head()

,MOVIL,PLACA,TIPO DE VEHÍCULO
0,200.0,H2H504,XL
1,201.0,AU0432,COMFORT
2,202.0,ATT007,COMFORT
3,203.0,BZB105,LITE
4,204.0,AWS428,COMFORT XL


In [13152]:
df_tipo_vehiculo.rename(columns={'TIPO DE VEHÍCULO':'Tipo_Vehiculo'}, inplace=True)
df_tipo_vehiculo.dropna(subset=['MOVIL'],inplace=True)
df_tipo_vehiculo['MOVIL']=df_tipo_vehiculo['MOVIL'].astype(float).astype(int).astype(str)
df_tipo_vehiculo.head()

,MOVIL,PLACA,Tipo_Vehiculo
0,200,H2H504,XL
1,201,AU0432,COMFORT
2,202,ATT007,COMFORT
3,203,BZB105,LITE
4,204,AWS428,COMFORT XL


In [13153]:
df_tipo_vehiculo.tail()

,MOVIL,PLACA,Tipo_Vehiculo
3237,4145,Csh416,COMFORT XL
3238,4128,CNB566,COMFORT XL
3239,3797,CJQ369,COMFORT XL
3240,4131,Cvr242,COMFORT XL
3241,16,MOVIL CONTINGENCIA,0


In [13154]:
len(df_tipo_vehiculo)

3239

In [13155]:
df_tipo_vehiculo.dropna(subset=['MOVIL'], inplace=True)

In [13156]:
df_tipo_vehiculo['anho'] = anho
df_tipo_vehiculo['LIQ'] = liquidacion
df_tipo_vehiculo['id'] = df_tipo_vehiculo['anho'].astype(str) + df_tipo_vehiculo['LIQ'].astype(str) + df_tipo_vehiculo['MOVIL'].astype(float).astype(int).astype(str)
df_tipo_vehiculo.head()

,MOVIL,PLACA,Tipo_Vehiculo,anho,LIQ,id
0,200,H2H504,XL,2026,03,202603200
1,201,AU0432,COMFORT,2026,03,202603201
2,202,ATT007,COMFORT,2026,03,202603202
3,203,BZB105,LITE,2026,03,202603203
4,204,AWS428,COMFORT XL,2026,03,202603204


# Lectura Data

In [13157]:
if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_dscts = pd.read_excel(
            ruta_archivo, 
            sheet_name='MATRIZ', 
            header=4
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_dscts)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        print("Flag1")
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_dscts.columns =  df_dscts.columns.astype(str)
        df_dscts = df_dscts.loc[:, ~df_dscts.columns.str.contains('^Unnamed')]
        print("Flag2")
        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        #if columna_clave in df_dscts.columns:
        #    filas_antes = len(df_dscts)
        #    df_dscts = df_dscts.dropna(subset=[columna_clave])
        #    filas_despues = len(df_dscts)
        #    print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_dscts.columns if 'MOVIL' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_dscts[col].dtype == 'object':
                df_dscts[col] = (
                    df_dscts[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_dscts[col] = pd.to_numeric(df_dscts[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_dscts.head())
        print(df_dscts.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

🚀 Cargando archivo: C:/Users/Usuario/DIRECTO Dropbox/Frank Jeshúa Nieto Sanchez Concha/LIQUIDACION AEROPUERTO/2026/TODAS LAS FLOTAS/LIQ. 03 TODAS LAS FLOTAS DEL 12 ENERO AL 18 ENERO.xlsx
✅ Lectura inicial completada. Filas detectadas: 17438
Flag1
Flag2

📊 DATOS LISTOS:
    Movil N° Servicio                Fecha Cliente Unidad de Negocio  \
0  1410.0  B012-70046  2026-01-12 00:00:00    KUSI            ORIGEN   
1   794.0  B012-70048  2026-01-12 00:00:00    KUSI            ORIGEN   
2  3815.0  B012-70045  2026-01-12 00:00:00    KUSI            ORIGEN   
3  9656.0  B012-70044  2026-01-12 00:00:00    KUSI            ORIGEN   
4   879.0  B012-70043  2026-01-12 00:00:00    KUSI            ORIGEN   

  Tipo de Servicio Tipo de pago  Peaje  Monto a Liquidar  Total Servicio  \
0           TICKET      Crédito    0.0              60.0            60.0   
1           TICKET      Crédito    0.0              50.0            50.0   
2           TICKET      Crédito    0.0              75.0            7

In [13158]:
df_data = df_dscts.copy()

# 1. Limpiamos espacios basura en los títulos
df_data.columns = df_data.columns.str.strip()

# -----------------------------------------------------------------------------
# ✂️ AQUÍ CORTAMOS HASTA LA PRIMERA FILA VACÍA
# -----------------------------------------------------------------------------
# Definimos qué columna manda (si esta está vacía, cortamos ahí)
columna_clave = 'Movil' # Asegúrate de que el nombre sea exacto (Movil vs MOVIL)

# Verificamos si existe la columna para no dar error
if columna_clave in df_data.columns:
    # Creamos una máscara de vacíos (True donde es NaN, False donde hay dato)
    mask_vacios = df_data[columna_clave].isna()
    
    # Si hay al menos un vacío...
    if mask_vacios.any():
        # .idxmax() en una serie de True/False devuelve el índice del PRIMER True
        primer_indice_vacio = mask_vacios.idxmax()
        
        print(f"✂️ Cortando datos en la fila {primer_indice_vacio} (primera celda vacía encontrada).")
        
        # Mantenemos solo las filas ANTERIORES a ese índice
        df_data = df_data.iloc[:primer_indice_vacio]
# -----------------------------------------------------------------------------

# 2. Seleccionamos la columna manteniendo el formato de tabla
df_data = df_data[['Movil']]

# Mostramos resultados
print(f"Filas finales: {len(df_data)}")
df_data.tail() # Muestra el final para que verifiques que ya no hay basura

✂️ Cortando datos en la fila 15335 (primera celda vacía encontrada).
Filas finales: 15335


,Movil
15330,4049.0
15331,2777.0
15332,4086.0
15333,3948.0
15334,3976.0


In [13159]:
df_data['anho'] = anho
df_data['LIQ'] = liquidacion
df_data['id'] = df_data['anho'].astype(str) + df_data['LIQ'].astype(str) + df_data['Movil'].astype(float).astype(int).astype(str)
df_data['Movil'] = df_data['Movil'].astype(float).astype(int).astype(str)
df_data.head()

,Movil,anho,LIQ,id
0,1410,2026,03,2026031410
1,794,2026,03,202603794
2,3815,2026,03,2026033815
3,9656,2026,03,2026039656
4,879,2026,03,202603879


In [13160]:
antes = len(df_data)
df_data.drop_duplicates(inplace=True)
despues = len(df_data)
print(f"Se eliminaron {antes - despues} duplicados")

Se eliminaron 14588 duplicados


# Cargar SQL

In [13161]:
!pip install pyodbc


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13162]:
import pandas as pd
import pypyodbc as odbc
from sqlalchemy import create_engine, types
from sqlalchemy.engine import URL, create_engine

# Configuración de conexión
servername = 'LP-MP2P84ND-PE'
database = 'Liquidacion'
username = 'sa'
password = '@nalistaDirecto'
# El driver debe coincidir con el instalado en tu PC (ej: 'SQL Server' o 'ODBC Driver 17 for SQL Server')
driver = 'ODBC Driver 17 for SQL Server' 

# Creación de la URL de conexión para SQLAlchemy
connection_string = f"DRIVER={{{driver}}};SERVER={servername};DATABASE={database};UID={username};PWD={password}"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)


In [13163]:
# 1. Convertir nombres de columnas a string y limpiar espacios en blanco
df_liquidacion.columns = [str(c).strip() for c in df_liquidacion.columns]

# 2. Identificar y eliminar la columna '0', '0.0' y las 'Unnamed'
# También eliminamos cualquier columna que sea puramente numérica (como '1', '2', etc.)
cols_invalidas = [c for c in df_liquidacion.columns if c in ['0', '0.0', 'None', ''] or 'Unnamed' in c]
df_liquidacion = df_liquidacion.drop(columns=cols_invalidas)

# 3. Eliminar columnas que sean solo números (si las hay)
import re
df_liquidacion = df_liquidacion.loc[:, [not re.match(r'^\d+(\.0)?$', c) for c in df_liquidacion.columns]]

# 4. Asegurar que no haya duplicados en los nombres de columnas
df_liquidacion = df_liquidacion.loc[:, ~df_liquidacion.columns.duplicated()]

print(f"✅ Columnas finales en df_liquidacion: {df_liquidacion.columns.tolist()}")

✅ Columnas finales en df_liquidacion: ['Plazas', 'Movil', 'Conductor', 'Propetario', 'Produccion_Bruta_Corporativos', 'Comision_Directo_Corporativos', 'Produccion_Neta_Conductor_Corporativos', 'Produccion_Bruta_Nexus', 'Comision_Td_Nexus', 'Produccion_Neta_Conductor_Nexus', 'Produccion_Bruta_Nexus_Efectivo', 'Comision_Td_Nexus_Efectivo', 'Total_Prod_Bruta_Cred_Y_Efect_Conductor', 'Total_Prod_Bruta_Credito_Conductor', 'Total_Prod_Bruta_Credito_Conductor_1', 'Total_Comision_Td_Credito', 'Produccion_Neta_Conductor_Credito', 'Membresia_Semanal', 'Asitencia', 'Urbano_Latam_Van', 'Desact', 'Permiso_Especial', 'Cupon_De_Permiso', 'Membresia_Final', 'Liquidacion_Anterior', 'Peaje_Nexus', 'Bono_Traslado', 'Peaje_Nexus_Wally', 'Otras_Rep', 'Total_Reposiciones', 'Liquidacion_Anterior_Proaccion', 'Liquidacion_Anterior_Transportes', 'Comision_Urbano', 'Fondo_Individual', 'Dscto_Activos', 'Dscto_Regularizacion', 'Dscto_Menbresia_Pex', 'Total_Descuentos', 'Pago_A_Conductor', 'Rxh', 'Monto_A_Pagar', '

In [13164]:
# 1. Asegurar que las columnas de montos sean FLOAT
#cols_numericas = ['MOVIL', 'Suma_de_Pago', 'TARIFA_BRUTA', 'Nro_Vale_SVS']
for col in cols_numericas:
    if col in df_dscts_tot.columns:
        df_dscts_tot[col] = pd.to_numeric(df_dscts_tot[col], errors='coerce').fillna(0)

# 2. El problema del "varchar a float": 'anho' y 'LIQ'
# Si en SQL son números, quítales las comillas convirtiéndolos a numeric
df_dscts_tot['anho'] = pd.to_numeric(df_dscts_tot['anho'], errors='coerce').fillna(0)
df_dscts_tot['LIQ'] = pd.to_numeric(df_dscts_tot['LIQ'], errors='coerce').fillna(0)

# 3. La columna 'id' suele ser el problema por ser muy larga
# Es mejor enviarla siempre como TEXTO (Varchar) para que SQL no intente calcularla
df_dscts_tot['id'] = df_dscts_tot['id'].astype(str)

In [13165]:
# 1. Asegurar nombres como string y limpiar espacios
df_no_pagados_total.columns = [str(c).strip() for c in df_no_pagados_total.columns]

# 2. Filtrar columnas: Quitamos el '.', '0', 'Unnamed' y cualquier cosa que no sea texto útil
columnas_validas = [
    c for c in df_no_pagados_total.columns 
    if c not in ['.', '0', '0.0', 'nan', 'None', ''] 
    and 'Unnamed' not in c
]

df_no_pagados_total = df_no_pagados_total[columnas_validas]

# 3. Limpieza extra: Eliminar cualquier columna que SOLO tenga símbolos o números
# Esto borrará columnas que se llamen ".", ",", "-", etc.
import re
df_no_pagados_total = df_no_pagados_total.loc[:, [not re.match(r'^[\W\d_]+$', c) for c in df_no_pagados_total.columns]]

print(f"✅ Columnas limpias para No Pagados: {df_no_pagados_total.columns.tolist()}")

✅ Columnas limpias para No Pagados: ['MOVIL', 'MONTO_EMITIR_PROACCION', 'MONTO_EMITIR_TRANSPORTES', 'MONTO_PAGAR_PROACCION', 'DETALLE_PROACCION', 'MONTO_PAGAR_TRANSPORTES', 'DETALLE_TRANSPORTE', 'anho', 'LIQ', 'id']


In [13166]:

df_dscts_tot['FECHA'] = pd.to_datetime(df_dscts_tot['FECHA'], dayfirst=True, errors='coerce')



In [13167]:
if 'Dscto_Regularizacion' in df_liquidacion.columns:
    df_liquidacion = df_liquidacion.drop(columns=['Dscto_Regularizacion'])
    print("✂️ Se omitió 'Dscto_Regularizacion' para que coincida con SQL.")

✂️ Se omitió 'Dscto_Regularizacion' para que coincida con SQL.


In [13168]:
# Forzamos a que estas columnas sean tratadas como string antes de subir
df_facturas['MOVIL'] = df_facturas['MOVIL'].astype(str).astype(float)
df_facturas['NRO_DE_FACTURA'] = df_facturas['NRO_DE_FACTURA'].astype(str)
df_facturas['NRO DE FACTURA.1'] = df_facturas['NRO DE FACTURA.1'].astype(str)

In [13169]:
cols_float = ['MOVIL', 'MONTO_A_EMITIR_PROACCION_TOTAL', 'MONTO_A_EMITIR_TRANSPORTES_TOTAL']

for col in cols_float:
    if col in df_facturas.columns:
        df_facturas[col] = pd.to_numeric(df_facturas[col], errors='coerce').fillna(0.0)

In [13170]:
df_facturas.columns

Index(['MOVIL', 'RUC', 'PROPETARIO', 'MONTO_A_EMITIR_PROACCION_TOTAL',
       'NRO_DE_FACTURA', 'MONTO_A_EMITIR_TRANSPORTES_TOTAL',
       'NRO DE FACTURA.1', 'DETALLE PROACCIÓN', 'OBSERVACIONES PROACCIÓN',
       'DETALLE_TRANSPORTES', 'OBSERVACIONES_TRANSPORTES',
       'MONTO_A_PAGAR_PROACCION', 'MONTO_A_PAGAR_TRANSPORTES', 'BANCO',
       'FECHA DE PAGO', 'SERVICIO CONTABLE', 'Factura / Rxh',
       'UNIDAD DE NEGOCIO', 'anho', 'LIQ', 'id'],
      dtype='object')

In [13171]:
#col_ordenadas = ['MOVIL', 'RUC', 'PROPETARIO', 'MONTO_A_EMITIR_PROACCION_TOTAL',
#'NRO_DE_FACTURA', 'MONTO_A_EMITIR_TRANSPORTES_TOTAL', 'NRO DE FACTURA.1', 'DETALLE_PROACCION',
#'OBSERVACIONES_PROACCION', 'DETALLE_TRANSPORTES', 'OBSERVACIONES_TRANSPORTES',
#'MONTO_A_PAGAR_PROACCION', 'MONTO_A_PAGAR_TRANSPORTES', 'BANCO', 'FECHA DE PAGO',
#'SERVICIO CONTABLE', 'Factura / Rxh', 'anho', 'LIQ', 'id']
#df_facturas = df_facturas[col_ordenadas]

In [13172]:
df_liquidacion = df_liquidacion.rename(columns={'Plazas': 'PLAZA'})
df_liquidacion.dtypes

PLAZA                                       object
Movil                                       object
Conductor                                   object
Propetario                                  object
Produccion_Bruta_Corporativos              float64
Comision_Directo_Corporativos              float64
Produccion_Neta_Conductor_Corporativos     float64
Produccion_Bruta_Nexus                     float64
Comision_Td_Nexus                          float64
Produccion_Neta_Conductor_Nexus            float64
Produccion_Bruta_Nexus_Efectivo            float64
Comision_Td_Nexus_Efectivo                 float64
Total_Prod_Bruta_Cred_Y_Efect_Conductor    float64
Total_Prod_Bruta_Credito_Conductor         float64
Total_Prod_Bruta_Credito_Conductor_1       float64
Total_Comision_Td_Credito                  float64
Produccion_Neta_Conductor_Credito          float64
Membresia_Semanal                          float64
Asitencia                                  float64
Urbano_Latam_Van               

In [13173]:
# 1. Limpiar la columna MOVIL (que es float en SQL)
# Convertimos a número y lo que sea texto ('CONTINGENCIA') se vuelve NaN
df_facturas['MOVIL'] = pd.to_numeric(df_facturas['MOVIL'], errors='coerce').astype(float)

# 2. Limpiar otras columnas numéricas según tu imagen de SQL
cols_a_limpiar = [
    'MONTO_A_EMITIR_PROACCION_TOTAL', 
    'MONTO_A_EMITIR_TRANSPORTES_TOTAL',
    'MONTO_A_PAGAR_PROACCION',
    'MONTO_A_PAGAR_TRANSPORTES',
    'FECHA DE PAGO'
]

for col in cols_a_limpiar:
    if col in df_facturas.columns:
        df_facturas[col] = pd.to_numeric(df_facturas[col], errors='coerce').fillna(0)

# 3. Eliminar filas donde el MOVIL sea nulo (como las de contingencia)
df_facturas = df_facturas.dropna(subset=['MOVIL'])

In [13174]:
df_facturas.dtypes

MOVIL                               float64
RUC                                 float64
PROPETARIO                           object
MONTO_A_EMITIR_PROACCION_TOTAL      float64
NRO_DE_FACTURA                       object
MONTO_A_EMITIR_TRANSPORTES_TOTAL    float64
NRO DE FACTURA.1                     object
DETALLE PROACCIÓN                    object
OBSERVACIONES PROACCIÓN              object
DETALLE_TRANSPORTES                  object
OBSERVACIONES_TRANSPORTES            object
MONTO_A_PAGAR_PROACCION             float64
MONTO_A_PAGAR_TRANSPORTES           float64
BANCO                                object
FECHA DE PAGO                       float64
SERVICIO CONTABLE                    object
Factura / Rxh                        object
UNIDAD DE NEGOCIO                    object
anho                                 object
LIQ                                  object
id                                   object
dtype: object

In [13175]:
#import sqlalchemy as sa

# 1. Obtener la lista REAL de columnas de la tabla en SQL
#inspector = sa.inspect(engine)
columnas_en_sql = [c['name'] for c in inspector.get_columns('tb_factura')]

# 2. Limpieza de datos críticos para evitar el error de 'varchar a float'
# Convertimos MOVIL y RUC a número; si dicen 'MOVIL CONTINGENCIA', se vuelven NaN
df_facturas['MOVIL'] = pd.to_numeric(df_facturas['MOVIL'], errors='coerce')
df_facturas['RUC'] = pd.to_numeric(df_facturas['RUC'], errors='coerce')

# 3. FILTRO MAESTRO: Solo nos quedamos con las columnas que existen en SQL
# Esto ignorará automáticamente 'DETALLE PROACCIÓN', 'UNIDAD DE NEGOCIO', etc.
df_para_sql = df_facturas[[c for c in df_facturas.columns if c in columnas_en_sql]].copy()

# 4. Eliminar filas donde el MOVIL sea nulo (basura del Excel o contingencias)
df_para_sql = df_para_sql.dropna(subset=['MOVIL'])

# 5. Subida final
print(f"🚀 Subiendo {len(df_para_sql.columns)} columnas compatibles a tb_factura...")
df_para_sql.to_sql('tb_factura', con=engine, if_exists='append', index=False)

🚀 Subiendo 18 columnas compatibles a tb_factura...


108

In [13176]:
df_para_sql.head()

,MOVIL,RUC,PROPETARIO,MONTO_A_EMITIR_PROACCION_TOTAL,NRO_DE_FACTURA,MONTO_A_EMITIR_TRANSPORTES_TOTAL,NRO DE FACTURA.1,DETALLE_TRANSPORTES,OBSERVACIONES_TRANSPORTES,MONTO_A_PAGAR_PROACCION,MONTO_A_PAGAR_TRANSPORTES,BANCO,FECHA DE PAGO,SERVICIO CONTABLE,Factura / Rxh,anho,LIQ,id
0,127.0,1.044282e+10,SONAN KAGUE CHRISTIAN ALFONSO,967.5000,PP,0.0000,nan,NO PROCEDE,NO TIENE PRODUCCIÓN,966.6000,0.0000,INTERBANK,0.0,BÁSICO,PRACTIPAGO,2026,03,202603127
1,205.0,1.021144e+10,BAUTISTA MENDOZA RANDHOP ALFONSO,1266.1018,E001 - 876,145.7960,E001 - 877,PROCEDE,NaN,1028.7018,126.8960,BCP,0.0,No,No,2026,03,202603205
2,207.0,1.043415e+10,YENGLE GONZALES FRANKLIN BERNABÈ,522.4265,E001 - 772,82.9150,E001 - 773,PROCEDE,NaN,147.3712,32.9150,BBVA,0.0,No,No,2026,03,202603207
3,210.0,1.002890e+10,GUEVARA TORRES ANTONIO,933.1069,E001 - 777,295.5616,E001 - 778,PROCEDE,NaN,760.6069,37.9616,BCP,0.0,No,No,2026,03,202603210
4,213.0,1.044043e+10,RODRIGUEZ DONAYRE MIGUEL ANGEL,728.6127,PP,293.9333,PP,PROCEDE,NaN,490.6127,137.8333,BCP,0.0,BÁSICO,PRACTIPAGO,2026,03,202603213


In [13177]:
#Tablas a cargar
tb_data = {
    'tb_factura': df_facturas,
    'tb_descuentos': df_dscts_tot,
    'tb_matriz': df_matriz,
    'tb_reposiciones': df_reps_total,
    'tb_fondo_individual': df_fondo_individual,
    'tb_no_pagados': df_no_pagados_total,
    'tb_liquidacion': df_liquidacion,
    'tb_cupones': df_cupones,
    'tb_tipo_vehiculo': df_tipo_vehiculo,
    'tb_data': df_data
}

tb_data_aux = {
    'tb_factura': df_para_sql
}

In [13180]:
def cargar_multiples_tablas(tb_data, engine):
    for nombre_tabla, df in tb_data.items():
        try:
            print(f"⏳ Procesando tabla: {nombre_tabla}...")

            # --- CARGA A SQL SERVER ---
            df.to_sql(nombre_tabla, con=engine, if_exists='append', index=False, chunksize=20000)
            
            print(f"Tabla '{nombre_tabla}' subida con éxito ({len(df)} filas).")
            
        except Exception as e:
            print(f"Error al subir la tabla '{nombre_tabla}': {e}")


cargar_multiples_tablas(tb_data_aux, engine)

⏳ Procesando tabla: tb_factura...
Tabla 'tb_factura' subida con éxito (920 filas).


In [13179]:
engine.dispose()